In [15]:
import os,string
import warnings
import scipy,re,nltk
import collections
import numpy as np
import pandas as pd
from unicodedata import normalize
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tqdm.notebook import tqdm as tqdm
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from transformers import *
from scipy import spatial
import plotly.express as px
from scipy.spatial import distance
from nltk.stem import SnowballStemmer
from sklearn.metrics import jaccard_score
from sklearn.model_selection import train_test_split

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import plotly.express as px
import plotly.graph_objects as go
import sklearn.metrics as metrics
from nltk.translate.bleu_score import sentence_bleu
from difflib import SequenceMatcher

from numpy import trapz
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import xgboost as xg
from nltk.translate.chrf_score import sentence_chrf
from nltk.translate.gleu_score import sentence_gleu
from nltk.translate.meteor_score import meteor_score
from sklearn.neural_network import MLPClassifier, MLPRegressor

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
%config InlineBackend.figure_format = 'retina'

### Upload Data

In [320]:
score_cs_en = pd.read_csv(os.path.join(os.getcwd(),'cs-en/scores.csv'))
display(score_cs_en.head())
score_de_en = pd.read_csv(os.path.join(os.getcwd(),'de-en/scores.csv'))
display(scores_de_en.head())
score_ru_en = pd.read_csv(os.path.join(os.getcwd(),'ru-en/scores.csv'))
display(score_ru_en.head())
score_zh_en = pd.read_csv(os.path.join(os.getcwd(),'zh-en/scores.csv'))
display(score_zh_en.head())
score_en_fi = pd.read_csv(os.path.join(os.getcwd(),'en-fi/scores.csv'))
display(score_en_fi.head())
score_en_zh = pd.read_csv(os.path.join(os.getcwd(),'en-zh/scores.csv'))
display(score_en_zh.head())

,source,reference,translation,z-score,avg-score,annotators
0,Uchopíte pak zbraň mezi své předloktí a rameno...,You will then grab the weapon between your for...,You then grasp the gun between your forearm an...,-0.675383,60.000000,3
1,"Ale je-li New York změna, pak je to také znovu...","But if New York is changed, then it's also a r...","But if New York is change, it is also reinvent...",-0.829403,44.000000,2
2,"Dlouho a intenzivně jsem během léta přemýšlel,...",I have been thinking over and over again over ...,I have thought long and hard over the course o...,0.803185,96.500000,2
3,"Najdou si jiný způsob, jak někde podvádět.",They find another way to cheat somewhere.,They will find another way how to defraud others.,0.563149,90.500000,2
4,Zpráva o výměně v čele prezidentovy administra...,The report on the replacement of the president...,The news of the replacement at the top of the ...,0.021549,74.666667,3


,source,reference,translation,z-score,avg-score,annotators
0,"Ihr Zeitlupentempo maßen sie, als sie vor Spit...",Her timeless pace measures them when they equi...,Their slow speed was measured by researchers o...,-0.345024,76.0,1
1,"Er sagte, dass die Bereiche ruhige Treffpunkte...",He said the areas offer quiet meeting points b...,He said the spaces provided calm meeting point...,0.903800,97.5,2
2,Für die Geschäftsleute an der B 27 ist es nur ...,"For businessmen at the B 27, it's only a small...",This is only a small consolation for businesse...,0.700503,94.0,1
3,Diese Fähigkeit sei möglicherweise angeboren o...,This ability may be born or developed with gen...,"This ability may be innate, or may develop as ...",-1.256572,51.5,2
4,Weil sie Wassertemperaturen um die sechs Grad ...,Because they prefer water temperatures around ...,They generally only come to the surface in win...,0.293909,87.0,2


,source,reference,translation,z-score,avg-score,annotators
0,В этом году крымчане получат уведомление на оп...,This year the Crimeans will receive a notice f...,"This year, residents of Crimea will receive a ...",0.878043,92.0,1
1,Энергетические компании находятся под давление...,Energy companies are under pressure due to low...,Energy companies are under pressure from lower...,0.511473,81.5,2
2,В бархатный сезон покупают туры в основном оди...,"In the velvet season, tours are mostly single ...","In the autumn season, tours are mainly purchas...",0.947866,94.0,1
3,Возле него на всякий случай стоит познавательн...,"Near him, just in case, there is a cognitive t...","Beside it, for good measure, there is an infor...",1.052601,97.0,1
4,Действительно ли Эфиопия находится на грани ра...,Is Ethiopia on the verge of schism?,Is Ethiopia about to crack?,0.738397,88.0,1


,source,reference,translation,z-score,avg-score,annotators
0,他性格活泼，这对英国赛马来说是好事，但是除此之外，他还是一位不可思议的骑师。,"His character is good for the British horse, b...",He's a lively character which is good for Brit...,0.625559,92.75,4
1,近日刚搬至旧金山的一位28岁厨师本周被发现死于当地一家商场的楼梯间。,"A 28 chef, who has just moved to San Francisco...",A 28-year-old chef who had recently moved to S...,0.550952,92.00,4
2,去年，有官员表示，胡克先生的团队所得出的结论是针对伊斯兰国炼油厂的空袭并未大幅削减恐怖组织的...,"Last year, officials said Mr. Hooker's team ha...","Last year, officials said, Mr. Hooker's team c...",0.540814,89.00,5
3,尤其值得玩味的是政府对于饥饿民众们的回应，比如总统市民赫伯特·胡佛“别人的事我可管不了”的态度。,It is particularly interesting to note the gov...,Especially savory are the accounts of the gove...,-0.793944,49.50,4
4,不过，从20世纪90年代至今，人类共进行了18次火星探测，而月球探测只进行了14次。,"However, from the 1990s to the present, human ...","However, ever since the 1990s, a total of 18 h...",0.046532,77.50,4


,source,reference,translation,z-score,avg-score,annotators
0,"You can turn yourself into a pineapple, a dog ...","Voit muuttaa itsesi ananasta, koirasta tai Roy...","Voit muuttaa itsesi ananakseksi, koiraksi tai ...",-0.286195,34.20,5
1,Also shot were three men: two 29-year-olds and...,Myös ammuttiin kolme miestä: kaksi 29-vuotiait...,Myös kolmea miestä ammuttiin: kahta 29-vuotias...,0.547076,58.40,5
2,The information is stored at the cash register...,Tiedot tallennetaan kassakoneisiin joka tapauk...,Tiedot kuitenkin tallentuvat kassoilla joka ta...,1.122476,74.60,5
3,Xinhua says that there were traces of hydrochl...,"Xinhua kertoo, että Xinyin näytteestä oli sunn...","Xinhua kertoo, että Xinyin sunnuntaina antamas...",0.383095,53.60,5
4,"MacDonald, who was brought on board CBC's comm...",Voitaisiin kuulla CBD: n kommenttitiimin toimi...,"MacDonaldin, joka tuli CBC:n selostajatiimiin ...",-0.493065,32.25,4


,source,reference,translation,z-score,avg-score,annotators
0,"""In the GISS model's simulation, Venus' slow s...",GSIS的科学家AnthonyDelGenio在新闻稿中解释说：“在GISS模型的模拟模型中...,戈达德太空研究所科学家安东尼·德尔·杰尼奥在新闻发布会上解释说：“在戈达德太空研究所的模型模...,-1.171867,50.0,1
1,Ai Yanhan of China in the Women's 4 x 200m Fre...,中国在英国女性4x200mFreestreyWTE中的最后被称为：“中国14岁的孩子从球下降...,参加女子4x200米自由泳接力赛决赛的中国小将艾衍含被这样描述：“那名14岁的中国小姑娘犯了...,-2.255403,26.5,2
2,"Then came 2012, when nothing much went right f...",然后来到2012年，当她和她的队友们没有什么好处。,2012年，她和她的队友都不被看好。,-2.508996,21.0,1
3,"Since last year, Guodian Group has exported a ...",自去年以来，GoudianGroup从南非通过南非港口出口了163套风力发电项目。,自去年以来，国电集团共计有163套风电项目陆续从连云港港出口南非。,-2.416780,23.0,1
4,"Some alleged that the Kempinski hotel simply ""...","一些人指称，Kempinski旅馆只是""被捕""，以满足阿拉伯客户的要求。",有人认为凯宾斯基酒店简直是为了满足阿拉伯客户的要求而“卑躬屈膝”。,-1.489676,45.0,7


In [456]:
from nltk.translate.chrf_score import sentence_chrf
import jieba

def punctuation_(word):
    return re.sub('[\“\”\ "\-\'`~!@#$%^&*()_|+=?;:,.<>\{\}\[\]\\\/]','', word)

def preprocessing(dataframe, column, lng):
    processed_corpus = []
    stop_words = set((stopwords.words(lng)))
    table = str.maketrans('', '', string.punctuation)
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    lem = WordNetLemmatizer()
    
    for i in tqdm(range(len(dataframe))):
        text = dataframe[column].iloc[i]
        if isinstance(text,float): text=''

        # normalize unicode characters
        text = normalize('NFD', text).encode('ascii', 'ignore')
        text = text.decode('UTF-8')

        # tokenize on white space
        text = text.split()

        # convert to lowercase
        text = [word.lower() for word in text]

        # remove punctuation from each token
        text = [word.translate(table) for word in text]
        text = [punctuation_(word) for word in text]

        # remove non-printable chars form each token
        text = [re_print.sub('', w) for w in text]
        
        # remove words not in glove embedding list
        #text = [word for word in text if word in glove_vocab ]

        # remove tokens with numbers in them
        text = [word for word in text if word.isalpha()]
        
        # remove stopwords
        text = [w for w in text if w not in stop_words]
        
        #Lemmatisation
        text = [lem.lemmatize(word) for word in text if not word in stop_words]

        # Join text back together
        text = " ".join(text)
        processed_corpus.append(text)
        
    return processed_corpus

def preprocessing_chinese(dataframe, column,lng):
    processed_corpus = []
    stop_words = set((stopwords.words(lng)))
    table = str.maketrans('', '', string.punctuation)
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    lem = WordNetLemmatizer()
    
    for i in tqdm(range(len(dataframe))):
        text = dataframe[column].iloc[i]
        
        # normalize unicode characters
        #text = normalize('NFD', text).encode('ascii', 'ignore')
        #text = text.decode('UTF-8')

        # tokenize on white space
        #text = text.split()
        
        #JIEBA to tokenize
        text = list(jieba.cut(text))

        # convert to lowercase
        text = [word.lower() for word in text]

        # remove punctuation from each token
        text = [word.translate(table) for word in text]

        # remove non-printable chars form each token
        #text = [re_print.sub('', w) for w in text]

        # remove tokens with numbers in them
        text = [word for word in text if word.isalpha()]

        # Join text back together
        text = " ".join(text)
        processed_corpus.append(text)
        
    return processed_corpus

In [322]:
from string import punctuation
from pymystem3 import Mystem

def preprocessing_russian(dataframe, column, lng):
    mystem = Mystem()
    processed_corpus = []
    stop_words = stopwords.words('russian')
    table = str.maketrans('', '', string.punctuation)
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    lem = WordNetLemmatizer()
    
    for i in tqdm(range(len(dataframe))):    
        text = dataframe[column].iloc[i]
       
        # normalize unicode characters
        #text = normalize('NFD', text).encode('ascii', 'ignore')
        #text = text.decode('UTF-8')

        # remove non-printable chars form each token
        #text = [re_print.sub('', w) for w in text]

        # remove words not in glove embedding list
        #text = [word for word in text if word in glove_vocab ]
        
        #Lemmatisation
        #text = [lem.lemmatize(word) for word in text if not word in stop_words]
        text = mystem.lemmatize(text.lower())

        # remove stopwords & punctuation
        #text = [w for w in text if w not in stop_words]
        text = [word for word in text if word not in stop_words and word != " "  and word.strip() not in punctuation]
        
        # tokenize on white space
        #text = text.split()

        # remove tokens with numbers in them
        text = [word for word in text if word.isalpha()]

        # convert to lowercase
        text = [word.lower() for word in text]

        # remove punctuation from each token
        text = [word.translate(table) for word in text]
        
        # Join text back together
        text = " ".join(text)
        processed_corpus.append(text)

    return processed_corpus

In [323]:
lng=['german','english','czech','russian','finnish','chinese-zh']
de_en_src_proc=preprocessing(score_de_en, 'source', lng[0])
de_en_ref_proc=preprocessing(score_de_en, 'reference', lng[1])
de_en_trn_proc=preprocessing(score_de_en, 'translation', lng[1])

cs_en_src_proc=preprocessing(score_cs_en, 'source',lng[2])
cs_en_ref_proc=preprocessing(score_cs_en, 'reference',lng[1])
cs_en_trn_proc=preprocessing(score_cs_en, 'translation',lng[1])

ru_en_src_proc=preprocessing_russian(score_ru_en, 'source',lng[3])
ru_en_ref_proc=preprocessing(score_ru_en, 'reference',lng[1])
ru_en_trn_proc=preprocessing(score_ru_en, 'translation',lng[1])

en_fi_src_proc=preprocessing(score_en_fi, 'source',lng[1])
en_fi_ref_proc=preprocessing(score_en_fi, 'reference',lng[4])
en_fi_trn_proc=preprocessing(score_en_fi, 'translation',lng[4])

zh_en_src_proc=preprocessing_chinese(score_zh_en, 'source',lng[5])
zh_en_ref_proc=preprocessing(score_zh_en, 'reference',lng[1])
zh_en_trn_proc=preprocessing(score_zh_en, 'translation',lng[1])

en_zh_src_proc=preprocessing(score_en_zh, 'source',lng[1])
en_zh_ref_proc=preprocessing_chinese(score_en_zh, 'reference',lng[5])
en_zh_trn_proc=preprocessing_chinese(score_en_zh, 'translation',lng[5])

  0%|          | 0/21704 [00:00<?, ?it/s]

  0%|          | 0/21704 [00:00<?, ?it/s]

  0%|          | 0/21704 [00:00<?, ?it/s]

  0%|          | 0/11585 [00:00<?, ?it/s]

  0%|          | 0/11585 [00:00<?, ?it/s]

...

  0%|          | 0/26419 [00:00<?, ?it/s]

  0%|          | 0/26419 [00:00<?, ?it/s]

  0%|          | 0/10221 [00:00<?, ?it/s]

  0%|          | 0/10221 [00:00<?, ?it/s]

  0%|          | 0/10221 [00:00<?, ?it/s]

### Data Preprocessing

In [327]:
dfp_cs_en = pd.DataFrame()
dfp_cs_en['source'] = cs_en_src_proc
dfp_cs_en['reference'] = cs_en_ref_proc
dfp_cs_en['translation'] = cs_en_trn_proc
dfp_cs_en['avg-score'] = score_cs_en['avg-score']
dfp_cs_en['z-score'] = score_cs_en['z-score']
display(dfp_cs_en.head())

dfp_de_en = pd.DataFrame()
dfp_de_en['source'] = de_en_src_proc
dfp_de_en['reference'] = de_en_ref_proc
dfp_de_en['translation'] = de_en_trn_proc
dfp_de_en['avg-score'] = score_de_en['avg-score']
dfp_de_en['z-score'] = score_de_en['z-score']
display(dfp_de_en.head())

dfp_ru_en = pd.DataFrame()
dfp_ru_en['source'] = ru_en_src_proc
dfp_ru_en['reference'] = ru_en_ref_proc
dfp_ru_en['translation'] = ru_en_trn_proc
dfp_ru_en['avg-score'] = score_ru_en['avg-score']
dfp_ru_en['z-score'] = score_ru_en['z-score']
display(dfp_ru_en.head())

dfp_en_fi = pd.DataFrame()
dfp_en_fi['source'] = en_fi_src_proc
dfp_en_fi['reference'] = en_fi_ref_proc
dfp_en_fi['translation'] = en_fi_trn_proc
dfp_en_fi['avg-score'] = score_en_fi['avg-score']
dfp_en_fi['z-score'] = score_en_fi['z-score']
display(dfp_en_fi.head())

dfp_zh_en = pd.DataFrame()
dfp_zh_en['source'] = zh_en_src_proc
dfp_zh_en['reference'] = zh_en_ref_proc
dfp_zh_en['translation'] = zh_en_trn_proc
dfp_zh_en['avg-score'] = score_zh_en['avg-score']
dfp_zh_en['z-score'] = score_zh_en['z-score']
display(dfp_zh_en.head())

dfp_en_zh = pd.DataFrame()
dfp_en_zh['source'] = en_zh_src_proc
dfp_en_zh['reference'] = en_zh_ref_proc
dfp_en_zh['translation'] = en_zh_trn_proc
dfp_en_zh['avg-score'] = score_en_zh['avg-score']
dfp_en_zh['z-score'] = score_en_zh['z-score']
display(dfp_en_zh.head())

,source,reference,translation,avg-score,z-score
0,uchopite zbran sve predlokti a rameno tesne pr...,grab weapon forearm shoulder hit face free elbow,grasp gun forearm shoulder hitting face free e...,60.000000,-0.675383
1,jeli new york zmena take znovuobjevovani,new york changed also rediscovery,new york change also reinvention,44.000000,-0.829403
2,dlouho a intenzivne behem leta premyslel mohu ...,thinking summer improve give depth need get hi...,thought long hard course summer might improve ...,96.500000,0.803185
3,najdou si jiny zpusob nekde podvadet,find another way cheat somewhere,find another way defraud others,90.500000,0.563149
4,zprava o vymene v cele prezidentovy administra...,report replacement president administration he...,news replacement top president office come sur...,74.666667,0.021549


,source,reference,translation,avg-score,z-score
0,zeitlupentempo maen spitzbergen sechs tiere se...,timeless pace measure equipped six animal broa...,slow speed measured researcher svalbard fitted...,76.0,-0.345024
1,sagte bereiche ruhige treffpunkte fluchtlingen...,said area offer quiet meeting point refugee vo...,said space provided calm meeting point refugee...,97.5,0.903800
2,fur geschaftsleute b kleiner trost fur kunden ...,businessmen b small consolation customer rotte...,small consolation business located along b roa...,94.0,0.700503
3,fahigkeit sei moglicherweise angeboren entwick...,ability may born developed gender maturity,ability may innate may develop animal reach se...,51.5,-1.256572
4,wassertemperaturen sechs grad celsius bevorzug...,prefer water temperature around six degree cel...,generally come surface winter prefer water tem...,87.0,0.293909


,source,reference,translation,avg-score,z-score
0,год крымчанин получать уведомление оплата тран...,year crimeans receive notice payment transport...,year resident crimea receive notice pay transp...,92.0,0.878043
1,энергетический компания находиться давление по...,energy company pressure due lowered price crud...,energy company pressure lower crude oil price ...,81.5,0.511473
2,бархатный сезон покупать тур основной одиночны...,velvet season tour mostly single traveler couple,autumn season tour mainly purchased individual...,94.0,0.947866
3,возле всякий случай стоять познавательный табл...,near case cognitive tablet reminding internati...,beside good measure informative plaque remindi...,97.0,1.052601
4,действительно эфиопия находиться грань раскол,ethiopia verge schism,ethiopia crack,88.0,0.738397


,source,reference,translation,avg-score,z-score
0,turn pineapple dog character befitting roy lic...,voit muuttaa itsesi ananasta koirasta roy lich...,voit muuttaa itsesi ananakseksi koiraksi hahmo...,34.20,-0.286195
1,also shot three men two one old,myos ammuttiin kolme miesta kaksi yksi,myos kolmea miesta ammuttiin kahta yhta,58.40,0.547076
2,information stored cash register anyway whethe...,tiedot tallennetaan kassakoneisiin tapauksessa...,tiedot kuitenkin tallentuvat kassoilla tapauks...,74.60,1.122476
3,xinhua say trace hydrochlorothiazide diuretic ...,xinhua kertoo etta xinyin naytteesta sunnuntai...,xinhua kertoo etta xinyin sunnuntaina antamast...,53.60,0.383095
4,macdonald brought board cbc commentary team pr...,voitaisiin kuulla cbd n kommenttitiimin toimit...,macdonaldin tuli cbcn selostajatiimiin tuomaan...,32.25,-0.493065


,source,reference,translation,avg-score,z-score
0,他 性格 活泼 这 对 英国 赛马 来说 是 好事 但是 除此之外 他 还是 一位 不可思议...,character good british horse addition incredib...,he lively character good british racing he inc...,92.75,0.625559
1,近日 刚 搬 至 旧金山 的 一位 岁 厨师 本周 被 发现 死 于 当地 一家 商场 的 楼梯间,chef moved san francisco found dead stair loca...,chef recently moved san francisco found dead s...,92.00,0.550952
2,去年 有 官员 表示 胡克 先生 的 团队 所 得出 的 结论 是 针对 伊斯兰 国 炼油厂...,last year official said mr hooker team conclud...,last year official said mr hooker team conclud...,89.00,0.540814
3,尤其 值得 玩味 的 是 政府 对于 饥饿 民众 们 的 回应 比如 总统 市民 赫伯特 胡...,particularly interesting note government respo...,especially savory account government response ...,49.50,-0.793944
4,不过 从 世纪 年代 至今 人类 共 进行 了 次 火星 探测 而 月球 探测 只 进行 了 次,however present human being carried mar probe ...,however ever since total human exploration car...,77.50,0.046532


,source,reference,translation,avg-score,z-score
0,gi model simulation venus slow spin expose day...,gsis 的 科学家 anthonydelgenio 在 新闻稿 中 解释 说 在 giss...,戈达德 太空 研究所 科学家 安东尼 德尔 杰 尼奥 在 新闻 发布会 上 解释 说 在 戈...,50.0,-1.171867
1,ai yanhan china woman x freestyle relay final ...,中国 在 英国 女性 中 的 最后 被 称为 中国 岁 的 孩子 从球 下降 到 了 孩子,参加 女子 米 自由泳 接力赛 决赛 的 中国 小将 艾衍 含 被 这样 描述 那名 岁 的...,26.5,-2.255403
2,came nothing much went right teammate,然后 来到 年 当 她 和 她 的 队友 们 没有 什么 好处,年 她 和 她 的 队友 都 不 被 看好,21.0,-2.508996
3,since last year guodian group exported total s...,自 去年 以来 goudiangroup 从 南非 通过 南非 港口 出口 了 套 风力 发...,自 去年 以来 国电 集团 共计 有 套 风电 项目 陆续 从 连云港 港 出口 南非,23.0,-2.416780
4,alleged kempinski hotel simply caved alleged d...,一些 人 指称 kempinski 旅馆 只是 被捕 以 满足 阿拉伯 客户 的 要求,有人 认为 凯宾斯基 酒店 简直 是 为了 满足 阿拉伯 客户 的 要求 而 卑躬屈膝,45.0,-1.489676


In [325]:
def MED(token1, token2):
    a,b,c = 0,0,0
    distances = np.zeros((len(token1) + 1, len(token2) + 1))
    for t1 in range(len(token1) + 1): distances[t1][0] = t1
    for t2 in range(len(token2) + 1): distances[0][t2] = t2
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                if (a <= b and a <= c): distances[t1][t2] = a + 1
                elif (b <= a and b <= c): distances[t1][t2] = b + 1
                else: distances[t1][t2] = c + 1
                    
    return distances[len(token1)][len(token2)]

In [502]:
def get_metrics(df,columns):
    metrics={}
    metrics.update({'cossim':[],'jacc':[],'dice':[],'bleu1':[],'bleu2':[],'bleu3':[],'bleu4':[]})
    metrics.update({'difflib':[],'fz':[],'med':[],'chrf':[],'gleu':[],'meteor':[]})
    for i in tqdm(range(len(df))):
        s1 = df[columns[1]].iloc[i] # ref
        s2 = df[columns[2]].iloc[i] # hyp
        if len(s1.split()) == 0 | len(s2.split()) == 0:
            metrics['cossim'].append(0)
            metrics['jacc'].append(0)
            metrics['dice'].append(0)
            metrics['bleu1'].append(0)
            metrics['bleu2'].append(0)
            metrics['bleu3'].append(0)
            metrics['bleu4'].append(0)
            metrics['difflib'].append(0)
            metrics['fz'].append(0)
            metrics['med'].append(0)
            metrics['chrf'].append(0)
            metrics['gleu'].append(0)
            metrics['meteor'].append(0)
        else:    
            cv = CountVectorizer()
            try:
                v = cv.fit_transform([s1,s2])
                metrics['cossim'].append(1 - spatial.distance.cosine(v.toarray()[0],v.toarray()[1]))
                metrics['jacc'].append(jaccard_score(v.toarray()[0],v.toarray()[1], average = 'weighted'))
                metrics['dice'].append(1 - distance.dice(v.toarray()[0],v.toarray()[1]))
                metrics['bleu1'].append(sentence_bleu([s1.split()], s2.split(),weights=(1, 0, 0, 0))) # ref, hyp
                metrics['bleu2'].append(sentence_bleu([s1.split()], s2.split(),weights=(0.5, 0.5, 0, 0))) # ref, hyp
                metrics['bleu3'].append(sentence_bleu([s1.split()], s2.split(),weights=(0.33, 0.33, 0.33, 0))) # ref, hyp
                metrics['bleu4'].append(sentence_bleu([s1.split()], s2.split(),weights=(0.25, 0.25, 0.25, 0.25))) # ref, hyp
                metrics['difflib'].append(SequenceMatcher(None, s1, s2).ratio())
                metrics['fz'].append(fuzz.token_set_ratio(s1,s2))
                metrics['med'].append(MED(s1,s2))
                metrics['chrf'].append(sentence_chrf(s1,s2))
                metrics['gleu'].append(sentence_gleu([s1.split()], s2.split()))
                metrics['meteor'].append(meteor_score([s1], s2))
            except ValueError:
                metrics['cossim'].append(0)
                metrics['jacc'].append(0)
                metrics['dice'].append(0)
                metrics['bleu1'].append(0)
                metrics['bleu2'].append(0)
                metrics['bleu3'].append(0)
                metrics['bleu4'].append(0)
                metrics['difflib'].append(0)
                metrics['fz'].append(0)
                metrics['med'].append(0)
                metrics['chrf'].append(0)
                metrics['gleu'].append(0)
                metrics['meteor'].append(0)
                pass
    metrics=pd.DataFrame.from_dict(metrics).copy()
    for i in range(len(columns)): metrics.insert(i,columns[i],df[columns[i]])
    return metrics

### Getting Metrics 

For that (as we have seen before) we will use the CountVectorizer from sklearn with the following parameters:

- max_df = 0.8 : When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). This is to ensure that we only have words relevant to the context and not commonly used words.
- stop_words = "english": This parameter excludes words inside the default english stopwords list from sklearn.
- max_features = 10000: This parameter makes sure that we consider only the top 10000 most frequent words as features (everything else is discarded).
- ngram_range (1, 3): This parameter forces the skleanr CountVectorizer to create unigram features, bigram features and trigram features

In [331]:
cs_en_metrics=get_metrics(dfp_cs_en,dfp_cs_en.columns[:5])
display(cs_en_metrics.head())
de_en_metrics=get_metrics(dfp_de_en,dfp_de_en.columns[:5])
display(de_en_metrics.head())
ru_en_metrics=get_metrics(dfp_ru_en,dfp_ru_en.columns[:5])
display(ru_en_metrics.head())
zh_en_metrics=get_metrics(dfp_zh_en,dfp_zh_en.columns[:5])
display(zh_en_metrics.head())
en_fi_metrics=get_metrics(dfp_en_fi,dfp_en_fi.columns[:5])
display(en_fi_metrics.head())
en_zh_metrics=get_metrics(dfp_en_zh,dfp_en_zh.columns[:5])
display(en_zh_metrics.head())

  0%|          | 0/11585 [00:00<?, ?it/s]

,source,reference,translation,avg-score,z-score,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,uchopite zbran sve predlokti a rameno tesne pr...,grab weapon forearm shoulder hit face free elbow,grasp gun forearm shoulder hitting face free e...,60.000000,-0.675383,0.625000,0.330579,0.625000,0.625000,5.175492e-01,3.584414e-01,5.614022e-78,0.857143,84,11.0,0.723854,0.346154,0.864796
1,jeli new york zmena take znovuobjevovani,new york changed also rediscovery,new york change also reinvention,44.000000,-0.829403,0.600000,0.306122,0.600000,0.600000,3.872983e-01,1.594901e-102,9.283143e-155,0.800000,80,10.0,0.553149,0.285714,0.793750
2,dlouho a intenzivne behem leta premyslel mohu ...,thinking summer improve give depth need get hi...,thought long hard course summer might improve ...,96.500000,0.803185,0.623610,0.246094,0.608696,0.500000,3.396831e-01,2.159628e-01,3.824527e-78,0.709220,87,36.0,0.612678,0.220000,0.905783
3,najdou si jiny zpusob nekde podvadet,find another way cheat somewhere,find another way defraud others,90.500000,0.563149,0.600000,0.306122,0.600000,0.600000,5.477226e-01,4.677351e-01,6.868092e-78,0.730159,76,11.0,0.516879,0.428571,0.588889
4,zprava o vymene v cele prezidentovy administra...,report replacement president administration he...,news replacement top president office come sur...,74.666667,0.021549,0.333333,0.120000,0.333333,0.333333,8.612150e-155,6.191520e-204,1.384293e-231,0.485149,61,72.0,0.321365,0.095238,0.208333


  0%|          | 0/21704 [00:00<?, ?it/s]

,source,reference,translation,avg-score,z-score,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,zeitlupentempo maen spitzbergen sechs tiere se...,timeless pace measure equipped six animal broa...,slow speed measured researcher svalbard fitted...,76.0,-0.345024,0.235702,0.071111,0.235294,0.222222,0.166667,9.142055e-103,6.089710e-155,0.492308,54,49.0,0.348880,0.100000,0.389660
1,sagte bereiche ruhige treffpunkte fluchtlingen...,said area offer quiet meeting point refugee vo...,said space provided calm meeting point refugee...,97.5,0.903800,0.625000,0.330579,0.625000,0.625000,0.517549,4.505656e-01,3.655552e-01,0.752294,81,15.0,0.643563,0.423077,0.864796
2,fur geschaftsleute b kleiner trost fur kunden ...,businessmen b small consolation customer rotte...,small consolation business located along b roa...,94.0,0.700503,0.377964,0.124260,0.375000,0.400000,0.210819,1.067587e-102,6.848991e-155,0.517986,76,54.0,0.667306,0.147059,0.384909
3,fahigkeit sei moglicherweise angeboren entwick...,ability may born developed gender maturity,ability may innate may develop animal reach se...,51.5,-1.256572,0.492366,0.099174,0.533333,0.333333,0.204124,1.045091e-102,6.739372e-155,0.653465,70,27.0,0.507798,0.133333,0.396825
4,wassertemperaturen sechs grad celsius bevorzug...,prefer water temperature around six degree cel...,generally come surface winter prefer water tem...,87.0,0.293909,0.818182,0.585799,0.818182,0.818182,0.756787,6.853034e-01,6.315552e-01,0.636943,92,57.0,0.829499,0.657895,0.896818


  0%|          | 0/17980 [00:00<?, ?it/s]

...

,source,reference,translation,avg-score,z-score,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,他 性格 活泼 这 对 英国 赛马 来说 是 好事 但是 除此之外 他 还是 一位 不可思议...,character good british horse addition incredib...,he lively character good british racing he inc...,92.75,0.625559,0.569803,0.388889,0.625000,0.555556,0.456435,3.135509e-01,5.072841e-78,0.747826,84,22.0,0.684272,0.300000,0.672222
1,近日 刚 搬 至 旧金山 的 一位 岁 厨师 本周 被 发现 死 于 当地 一家 商场 的 楼梯间,chef moved san francisco found dead stair loca...,chef recently moved san francisco found dead s...,92.00,0.550952,0.858116,0.625000,0.857143,0.818182,0.700649,6.050763e-01,4.832698e-01,0.897638,94,13.0,0.868150,0.552632,0.874587
2,去年 有 官员 表示 胡克 先生 的 团队 所 得出 的 结论 是 针对 伊斯兰 国 炼油厂...,last year official said mr hooker team conclud...,last year official said mr hooker team conclud...,89.00,0.540814,0.627293,0.268191,0.721311,0.537727,0.428074,3.643453e-01,3.119358e-01,0.442060,78,132.0,0.567639,0.341270,0.537559
3,尤其 值得 玩味 的 是 政府 对于 饥饿 民众 们 的 回应 比如 总统 市民 赫伯特 胡...,particularly interesting note government respo...,especially savory account government response ...,49.50,-0.793944,0.444750,0.190476,0.444444,0.427367,0.363192,2.844537e-01,4.624980e-78,0.640351,68,51.0,0.509731,0.240000,0.560364
4,不过 从 世纪 年代 至今 人类 共 进行 了 次 火星 探测 而 月球 探测 只 进行 了 次,however present human being carried mar probe ...,however ever since total human exploration car...,77.50,0.046532,0.526235,0.160000,0.571429,0.452419,0.213272,1.039811e-102,6.552766e-155,0.539474,73,45.0,0.446104,0.157895,0.341284


  0%|          | 0/6748 [00:00<?, ?it/s]

,source,reference,translation,avg-score,z-score,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,turn pineapple dog character befitting roy lic...,voit muuttaa itsesi ananasta koirasta roy lich...,voit muuttaa itsesi ananakseksi koiraksi hahmo...,34.20,-0.286195,0.500000,0.222222,0.500000,0.500000,4.082483e-01,2.787343e-01,4.640084e-78,0.731429,80,43.0,0.641148,0.264706,0.484000
1,also shot three men two one old,myos ammuttiin kolme miesta kaksi yksi,myos kolmea miesta ammuttiin kahta yhta,58.40,0.547076,0.500000,0.222222,0.500000,0.500000,1.054769e-154,7.077949e-204,1.531972e-231,0.519481,83,27.0,0.500398,0.166667,0.250000
2,information stored cash register anyway whethe...,tiedot tallennetaan kassakoneisiin tapauksessa...,tiedot kuitenkin tallentuvat kassoilla tapauks...,74.60,1.122476,0.316228,0.082645,0.307692,0.250000,7.458341e-155,5.630767e-204,1.288230e-231,0.661290,71,34.0,0.516654,0.076923,0.188679
3,xinhua say trace hydrochlorothiazide diuretic ...,xinhua kertoo etta xinyin naytteesta sunnuntai...,xinhua kertoo etta xinyin sunnuntaina antamast...,53.60,0.383095,0.683763,0.291667,0.666667,0.545455,4.045199e-01,3.349810e-01,2.596536e-01,0.707692,76,49.0,0.794253,0.315789,0.760135
4,macdonald brought board cbc commentary team pr...,voitaisiin kuulla cbd n kommenttitiimin toimit...,macdonaldin tuli cbcn selostajatiimiin tuomaan...,32.25,-0.493065,0.083333,0.022684,0.083333,0.076670,3.961780e-155,3.605175e-204,9.005790e-232,0.325123,53,68.0,0.332327,0.021739,0.038760


  0%|          | 0/10221 [00:00<?, ?it/s]

,source,reference,translation,avg-score,z-score,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,gi model simulation venus slow spin expose day...,gsis 的 科学家 anthonydelgenio 在 新闻稿 中 解释 说 在 giss...,戈达德 太空 研究所 科学家 安东尼 德尔 杰 尼奥 在 新闻 发布会 上 解释 说 在 戈...,50.0,-1.171867,0.243975,0.096154,0.285714,0.411765,0.223407,1.184963e-01,2.427154e-78,0.500000,55,58.0,0.138237,0.146154,0.382837
1,ai yanhan china woman x freestyle relay final ...,中国 在 英国 女性 中 的 最后 被 称为 中国 岁 的 孩子 从球 下降 到 了 孩子,参加 女子 米 自由泳 接力赛 决赛 的 中国 小将 艾衍 含 被 这样 描述 那名 岁 的...,26.5,-2.255403,0.259281,0.047619,0.320000,0.280000,0.108012,6.866195e-103,4.902402e-155,0.339286,43,46.0,0.079691,0.085106,0.187166
2,came nothing much went right teammate,然后 来到 年 当 她 和 她 的 队友 们 没有 什么 好处,年 她 和 她 的 队友 都 不 被 看好,21.0,-2.508996,0.288675,0.122449,0.250000,0.444491,0.382557,3.465067e-01,3.045753e-01,0.653846,69,15.0,0.230450,0.326087,0.463692
3,since last year guodian group exported total s...,自 去年 以来 goudiangroup 从 南非 通过 南非 港口 出口 了 套 风力 发...,自 去年 以来 国电 集团 共计 有 套 风电 项目 陆续 从 连云港 港 出口 南非,23.0,-2.416780,0.501745,0.140625,0.545455,0.500000,0.258199,1.712646e-01,3.208347e-78,0.365591,63,32.0,0.155391,0.189655,0.418046
4,alleged kempinski hotel simply caved alleged d...,一些 人 指称 kempinski 旅馆 只是 被捕 以 满足 阿拉伯 客户 的 要求,有人 认为 凯宾斯基 酒店 简直 是 为了 满足 阿拉伯 客户 的 要求 而 卑躬屈膝,45.0,-1.489676,0.381385,0.138408,0.380952,0.357143,0.331497,3.053772e-01,2.658484e-01,0.511628,51,28.0,0.272194,0.280000,0.380153


In [334]:
df_metr={}
df_metr.update({'cs_en_metrics':cs_en_metrics})
df_metr.update({'de_en_metrics':de_en_metrics})
df_metr.update({'ru_en_metrics':ru_en_metrics})
df_metr.update({'zh_en_metrics':zh_en_metrics})
df_metr.update({'en_fi_metrics':en_fi_metrics})
df_metr.update({'en_zh_metrics':en_zh_metrics})
df_metr.keys()

dict_keys(['cs_en_metrics', 'de_en_metrics', 'ru_en_metrics', 'zh_en_metrics', 'en_fi_metrics', 'en_zh_metrics'])

In [18]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from scipy.stats import kendalltau, pearsonr, spearmanr

class RegressionReport:
    def __init__(self):
        super().__init__()
        self.metrics = [Pearson(), Kendall(), Spearman()]

    def compute(self, x: np.array, y: np.array) -> float:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            return {metric.name: metric.compute(x, y) for metric in self.metrics}

class Kendall:
    def __init__(self):
        self.name = 'kendall'

    def compute(self, x: np.array, y: np.array) -> float:
        return torch.tensor(kendalltau(x, y)[0], dtype=torch.float32)
    
class Pearson:
    def __init__(self):
        self.name = 'pearson'

    def compute(self, x: np.array, y: np.array) -> torch.Tensor:
        return torch.tensor(pearsonr(x, y)[0], dtype=torch.float32)

class Spearman:
    def __init__(self):
        self.name = 'spearman'

    def compute(self, x: np.array, y: np.array) -> float:
        return torch.tensor(spearmanr(x, y)[0], dtype=torch.float32)

In [19]:
tokenizer = BertTokenizerFast.from_pretrained('setu4993/LaBSE')
model = BertModel.from_pretrained('setu4993/LaBSE')
model = model.eval()

In [22]:
def update_df_metrics(df):
    regression=RegressionReport()
    dfTemp=pd.DataFrame([])
    for idx,i in enumerate(df.columns[5:].tolist()):
        if len(df[df[i].isna()])!=0:
            index=df[df[i].isna()].index.tolist()
            dfNew=df[~df.index.isin(index)].copy()
            value=regression.compute(np.array(dfNew['z-score'].tolist()),np.array(dfNew[i].tolist()))
        else:
            value=regression.compute(np.array(df['z-score'].tolist()),np.array(df[i].tolist()))
        data=pd.DataFrame.from_dict(value, orient='index').rename(columns={0:i})
        dfTemp=data if idx==0 else pd.concat([dfTemp,data],axis=1)
    return dfTemp

In [346]:
def get_sentence_embedding(df,columns):
    dic=collections.defaultdict(list)
    for i in tqdm(range(len(df))):
        for j in columns:
            with torch.no_grad():
                dic[f'se_{j}'].append(model(**tokenizer(df[j].iloc[i], return_tensors='pt', padding=True)).pooler_output) #tokenizer & pooler -> embedding
            #p(float): the exponent value in the norm formulation. Default: 2
            dic[f'nm_{j}'].append(F.normalize(dic[f'se_{j}'][i], p=2)) #normalize
    return dic

In [24]:
def similarity(array,size,embd1,embd2,stype):
    for i in tqdm(range(size)):
        value=torch.matmul(embd1[i],embd2[i].transpose(0,1))
        array.append(value) if stype==0 else array.append(value.cpu().detach().numpy())
    return array

In [ ]:
""" def get_word_embedding(df,columns):
    for i in tqdm(range(len(df))):
        for j in columns:
            try:
                if i==0: 
                    df.insert(df.columns.size,'se_'+j,chr(32))
                    df.insert(df.columns.size,'nm_'+j,chr(32))
            except (ValueError, KeyError):
                pass
            df.loc[i,'se_'+j]=[[tokenizer(df[j].iloc[i], return_tensors='pt', padding=True)]]
            with torch.no_grad():
                #print(F.normalize(model(**df.loc[i,'se_'+j][0]).pooler_output, p=2))
                df.loc[i,'nm_'+j]=[F.normalize(model(**df.loc[i,'se_'+j][0]).pooler_output, p=2).numpy()] #normalization
        break
    return df
"""

### Getting Language Embeddings

In [360]:
df_embd={}
for i in ['cs_en','de_en','ru_en','zh_en','en_fi','en_zh']:
    df_embd[f'{i}_se']=get_sentence_embedding(df_metr[f'{i}_metrics'],df_metr[f'{i}_metrics'].columns[:3])
    #df_embd[f'{i}_se'][f'{i}_ref_tra']=similarity(df_embd[f'{i}_se'],size,df_embd[f'{i}_se']['nm_reference'],df_embd[f'{i}_se']['nm_translation'],0) #tensor
    #df_embd[f'{i}_se'][f'{i}_ref_tra_np']=similarity(df_embd[f'{i}_se'],size,df_embd[f'{i}_se']['nm_reference'],df_embd[f'{i}_se']['nm_translation'],1) #np
    #df_embd[f'{i}_se'][f'{i}_src_tra']=similarity(df_embd[f'{i}_se'],size,df_embd[f'{i}_se']['nm_source'],df_embd[f'{i}_se']['nm_translation'],0) #tensor
    #df_embd[f'{i}_se'][f'{i}_src_tra_np']=similarity(df_embd[f'{i}_se'],size,df_embd[f'{i}_se']['nm_source'],df_embd[f'{i}_se']['nm_translation'],1) #np
    display(pd.DataFrame.from_dict(df_embd[f'{i}_se']).head())

  0%|          | 0/21704 [00:00<?, ?it/s]

,se_source,nm_source,se_reference,nm_reference,se_translation,nm_translation
0,"[[tensor(0.0872), tensor(0.1289), tensor(-0.07...","[[tensor(0.0123), tensor(0.0182), tensor(-0.01...","[[tensor(0.1827), tensor(0.1072), tensor(-0.16...","[[tensor(0.0314), tensor(0.0184), tensor(-0.02...","[[tensor(0.0351), tensor(0.1712), tensor(-0.22...","[[tensor(0.0048), tensor(0.0235), tensor(-0.03..."
1,"[[tensor(0.3226), tensor(0.1930), tensor(-0.21...","[[tensor(0.0413), tensor(0.0247), tensor(-0.02...","[[tensor(0.4805), tensor(0.3181), tensor(-0.12...","[[tensor(0.0644), tensor(0.0426), tensor(-0.01...","[[tensor(0.4419), tensor(0.4019), tensor(-0.28...","[[tensor(0.0552), tensor(0.0502), tensor(-0.03..."
2,"[[tensor(0.1390), tensor(0.0803), tensor(-0.02...","[[tensor(0.0191), tensor(0.0110), tensor(-0.00...","[[tensor(0.1011), tensor(-0.0916), tensor(-0.1...","[[tensor(0.0119), tensor(-0.0108), tensor(-0.0...","[[tensor(0.1818), tensor(0.0492), tensor(0.004...","[[tensor(0.0222), tensor(0.0060), tensor(0.000..."
3,"[[tensor(0.6181), tensor(-0.2312), tensor(-0.2...","[[tensor(0.0750), tensor(-0.0281), tensor(-0.0...","[[tensor(0.6421), tensor(0.2579), tensor(0.139...","[[tensor(0.0608), tensor(0.0244), tensor(0.013...","[[tensor(0.2878), tensor(0.0839), tensor(-0.20...","[[tensor(0.0248), tensor(0.0072), tensor(-0.01..."
4,"[[tensor(-0.2659), tensor(-0.5713), tensor(-0....","[[tensor(-0.0230), tensor(-0.0494), tensor(-0....","[[tensor(-0.6701), tensor(-0.6523), tensor(-0....","[[tensor(-0.0528), tensor(-0.0514), tensor(-0....","[[tensor(-0.8285), tensor(-0.8042), tensor(-0....","[[tensor(-0.0671), tensor(-0.0652), tensor(-0...."


  0%|          | 0/17980 [00:00<?, ?it/s]

,se_source,nm_source,se_reference,nm_reference,se_translation,nm_translation
0,"[[tensor(0.2890), tensor(-0.1835), tensor(-0.0...","[[tensor(0.0308), tensor(-0.0195), tensor(-0.0...","[[tensor(0.2090), tensor(-0.3210), tensor(-0.1...","[[tensor(0.0235), tensor(-0.0361), tensor(-0.0...","[[tensor(0.3536), tensor(-0.1715), tensor(-0.0...","[[tensor(0.0446), tensor(-0.0216), tensor(-0.0..."
1,"[[tensor(-0.8353), tensor(-0.0206), tensor(-0....","[[tensor(-0.0616), tensor(-0.0015), tensor(-0....","[[tensor(-0.2903), tensor(-0.1708), tensor(-0....","[[tensor(-0.0291), tensor(-0.0171), tensor(-0....","[[tensor(-0.1665), tensor(-0.2992), tensor(-0....","[[tensor(-0.0165), tensor(-0.0297), tensor(-0...."
2,"[[tensor(0.0630), tensor(-0.1505), tensor(-0.2...","[[tensor(0.0063), tensor(-0.0151), tensor(-0.0...","[[tensor(0.1916), tensor(-0.1541), tensor(-0.1...","[[tensor(0.0245), tensor(-0.0197), tensor(-0.0...","[[tensor(-0.3777), tensor(-0.2233), tensor(-0....","[[tensor(-0.0414), tensor(-0.0245), tensor(-0...."
3,"[[tensor(-0.3388), tensor(-0.1261), tensor(0.4...","[[tensor(-0.0326), tensor(-0.0121), tensor(0.0...","[[tensor(-0.0850), tensor(0.1221), tensor(0.29...","[[tensor(-0.0123), tensor(0.0177), tensor(0.04...","[[tensor(-0.0864), tensor(0.3467), tensor(0.15...","[[tensor(-0.0104), tensor(0.0416), tensor(0.01..."
4,"[[tensor(-0.0708), tensor(-0.2183), tensor(-0....","[[tensor(-0.0079), tensor(-0.0243), tensor(-0....","[[tensor(0.3600), tensor(-0.2005), tensor(0.14...","[[tensor(0.0595), tensor(-0.0331), tensor(0.02...","[[tensor(0.1152), tensor(-0.1530), tensor(-0.0...","[[tensor(0.0161), tensor(-0.0214), tensor(-0.0..."


  0%|          | 0/26419 [00:00<?, ?it/s]

,se_source,nm_source,se_reference,nm_reference,se_translation,nm_translation
0,"[[tensor(0.0908), tensor(0.0690), tensor(-0.33...","[[tensor(0.0141), tensor(0.0107), tensor(-0.05...","[[tensor(0.4370), tensor(0.3091), tensor(-0.58...","[[tensor(0.0478), tensor(0.0338), tensor(-0.06...","[[tensor(0.3051), tensor(0.1884), tensor(-0.65...","[[tensor(0.0300), tensor(0.0185), tensor(-0.06..."
1,"[[tensor(-0.0573), tensor(0.0267), tensor(-0.4...","[[tensor(-0.0086), tensor(0.0040), tensor(-0.0...","[[tensor(-0.0109), tensor(0.0756), tensor(-0.5...","[[tensor(-0.0014), tensor(0.0098), tensor(-0.0...","[[tensor(0.1258), tensor(0.0677), tensor(-0.62...","[[tensor(0.0154), tensor(0.0083), tensor(-0.07..."
2,"[[tensor(-0.0905), tensor(-0.4161), tensor(-0....","[[tensor(-0.0126), tensor(-0.0580), tensor(-0....","[[tensor(-0.3697), tensor(-0.4027), tensor(-0....","[[tensor(-0.0391), tensor(-0.0426), tensor(-0....","[[tensor(-0.1339), tensor(-0.2386), tensor(-0....","[[tensor(-0.0176), tensor(-0.0314), tensor(-0...."
3,"[[tensor(0.2216), tensor(0.1149), tensor(-0.08...","[[tensor(0.0350), tensor(0.0182), tensor(-0.01...","[[tensor(0.4539), tensor(0.0824), tensor(0.119...","[[tensor(0.0469), tensor(0.0085), tensor(0.012...","[[tensor(0.4156), tensor(0.0874), tensor(0.253...","[[tensor(0.0512), tensor(0.0108), tensor(0.031..."
4,"[[tensor(-0.3444), tensor(-0.0829), tensor(-0....","[[tensor(-0.0429), tensor(-0.0103), tensor(-0....","[[tensor(-0.2309), tensor(0.1292), tensor(-0.3...","[[tensor(-0.0267), tensor(0.0149), tensor(-0.0...","[[tensor(-0.2055), tensor(-0.1974), tensor(-0....","[[tensor(-0.0192), tensor(-0.0184), tensor(-0...."


  0%|          | 0/6748 [00:00<?, ?it/s]

,se_source,nm_source,se_reference,nm_reference,se_translation,nm_translation
0,"[[tensor(-0.2699), tensor(0.2960), tensor(0.32...","[[tensor(-0.0402), tensor(0.0441), tensor(0.04...","[[tensor(-0.2688), tensor(0.2952), tensor(0.45...","[[tensor(-0.0334), tensor(0.0367), tensor(0.05...","[[tensor(-0.2143), tensor(0.1691), tensor(0.22...","[[tensor(-0.0233), tensor(0.0184), tensor(0.02..."
1,"[[tensor(0.6724), tensor(0.1199), tensor(-0.29...","[[tensor(0.0461), tensor(0.0082), tensor(-0.02...","[[tensor(-0.0357), tensor(0.0833), tensor(-0.6...","[[tensor(-0.0027), tensor(0.0064), tensor(-0.0...","[[tensor(-0.0965), tensor(0.3190), tensor(-0.5...","[[tensor(-0.0075), tensor(0.0249), tensor(-0.0..."
2,"[[tensor(-0.1329), tensor(-0.4284), tensor(-0....","[[tensor(-0.0136), tensor(-0.0439), tensor(-0....","[[tensor(-0.1316), tensor(-0.5441), tensor(-0....","[[tensor(-0.0151), tensor(-0.0623), tensor(-0....","[[tensor(-0.3406), tensor(-0.4870), tensor(-0....","[[tensor(-0.0411), tensor(-0.0587), tensor(-0...."
3,"[[tensor(0.1682), tensor(-0.1269), tensor(0.24...","[[tensor(0.0281), tensor(-0.0212), tensor(0.04...","[[tensor(0.0100), tensor(-0.3092), tensor(0.17...","[[tensor(0.0014), tensor(-0.0436), tensor(0.02...","[[tensor(0.0399), tensor(-0.1759), tensor(0.33...","[[tensor(0.0058), tensor(-0.0255), tensor(0.04..."
4,"[[tensor(-0.1331), tensor(0.3726), tensor(-0.0...","[[tensor(-0.0170), tensor(0.0476), tensor(-0.0...","[[tensor(0.0708), tensor(0.3066), tensor(-0.10...","[[tensor(0.0076), tensor(0.0330), tensor(-0.01...","[[tensor(-0.1116), tensor(0.1029), tensor(0.01...","[[tensor(-0.0156), tensor(0.0144), tensor(0.00..."


  0%|          | 0/10221 [00:00<?, ?it/s]

,se_source,nm_source,se_reference,nm_reference,se_translation,nm_translation
0,"[[tensor(-0.1999), tensor(-0.0196), tensor(0.2...","[[tensor(-0.0255), tensor(-0.0025), tensor(0.0...","[[tensor(-0.2073), tensor(-0.1861), tensor(0.4...","[[tensor(-0.0307), tensor(-0.0276), tensor(0.0...","[[tensor(-0.3724), tensor(-0.3125), tensor(0.2...","[[tensor(-0.0574), tensor(-0.0482), tensor(0.0..."
1,"[[tensor(-0.1346), tensor(0.0480), tensor(-0.0...","[[tensor(-0.0188), tensor(0.0067), tensor(-0.0...","[[tensor(-0.2019), tensor(0.1769), tensor(0.28...","[[tensor(-0.0266), tensor(0.0233), tensor(0.03...","[[tensor(0.1633), tensor(0.0884), tensor(0.230...","[[tensor(0.0319), tensor(0.0173), tensor(0.045..."
2,"[[tensor(-0.0117), tensor(-0.2462), tensor(-0....","[[tensor(-0.0010), tensor(-0.0217), tensor(-0....","[[tensor(-0.2387), tensor(-0.3755), tensor(-0....","[[tensor(-0.0319), tensor(-0.0502), tensor(-0....","[[tensor(0.1201), tensor(0.0478), tensor(-0.56...","[[tensor(0.0143), tensor(0.0057), tensor(-0.06..."
3,"[[tensor(0.1826), tensor(-0.2454), tensor(-0.0...","[[tensor(0.0209), tensor(-0.0281), tensor(-0.0...","[[tensor(0.0063), tensor(-0.1353), tensor(-0.3...","[[tensor(0.0009), tensor(-0.0203), tensor(-0.0...","[[tensor(0.0480), tensor(-0.1213), tensor(-0.1...","[[tensor(0.0079), tensor(-0.0199), tensor(-0.0..."
4,"[[tensor(0.2565), tensor(0.4963), tensor(-0.23...","[[tensor(0.0341), tensor(0.0661), tensor(-0.03...","[[tensor(0.2244), tensor(0.4024), tensor(-0.09...","[[tensor(0.0358), tensor(0.0641), tensor(-0.01...","[[tensor(-0.0394), tensor(0.1050), tensor(-0.2...","[[tensor(-0.0067), tensor(0.0179), tensor(-0.0..."


In [25]:
#dfBkp=dfp.copy()
de_en_sm=get_sentence_embedding(dfp,dfp.columns[:3])
columns=['nm_source','nm_reference','nm_translation','de_en_ref_tra','de_en_ref_tra_np','de_en_src_tra','de_en_src_tra_np']
de_en_sm[columns[3]]=similarity(de_en_sm[columns[2]],len(dfp),de_en_sm[columns[1]],de_en_sm[columns[2]],0) #tensor
de_en_sm[columns[4]]=similarity(de_en_sm[columns[3]],len(dfp),de_en_sm[columns[1]],de_en_sm[columns[2]],1) #np
de_en_sm[columns[5]]=similarity(de_en_sm[columns[5]],len(dfp),de_en_sm[columns[0]],de_en_sm[columns[2]],0) #tensor
de_en_sm[columns[6]]=similarity(de_en_sm[columns[6]],len(dfp),de_en_sm[columns[0]],de_en_sm[columns[2]],1) #np

  0%|          | 0/21702 [00:00<?, ?it/s]

  0%|          | 0/21702 [00:00<?, ?it/s]

  0%|          | 0/21702 [00:00<?, ?it/s]

In [33]:
pd.DataFrame.from_dict(de_en_sm).head()
#de_en_sm_bkp=de_en_sm.copy()

,se_source,nm_source,se_reference,nm_reference,se_translation,nm_translation,de_en_ref_tra,de_en_ref_tra_np,de_en_src_tra,de_en_src_tra_np
0,"[[tensor(0.0872), tensor(0.1289), tensor(-0.07...","[[tensor(0.0123), tensor(0.0182), tensor(-0.01...","[[tensor(0.1827), tensor(0.1072), tensor(-0.16...","[[tensor(0.0314), tensor(0.0184), tensor(-0.02...","[[tensor(0.0351), tensor(0.1712), tensor(-0.22...","[[tensor(0.0048), tensor(0.0235), tensor(-0.03...",[[tensor(0.7031)]],[[0.7030852]],[[tensor(0.5690)]],[[0.5689796]]
1,"[[tensor(0.3226), tensor(0.1930), tensor(-0.21...","[[tensor(0.0413), tensor(0.0247), tensor(-0.02...","[[tensor(0.4805), tensor(0.3181), tensor(-0.12...","[[tensor(0.0644), tensor(0.0426), tensor(-0.01...","[[tensor(0.4419), tensor(0.4019), tensor(-0.28...","[[tensor(0.0552), tensor(0.0502), tensor(-0.03...",[[tensor(0.9319)]],[[0.93192255]],[[tensor(0.6850)]],[[0.685023]]
2,"[[tensor(0.1390), tensor(0.0803), tensor(-0.02...","[[tensor(0.0191), tensor(0.0110), tensor(-0.00...","[[tensor(0.1011), tensor(-0.0916), tensor(-0.1...","[[tensor(0.0119), tensor(-0.0108), tensor(-0.0...","[[tensor(0.1818), tensor(0.0492), tensor(0.004...","[[tensor(0.0222), tensor(0.0060), tensor(0.000...",[[tensor(0.8144)]],[[0.8144466]],[[tensor(0.7505)]],[[0.7504732]]
3,"[[tensor(0.6181), tensor(-0.2312), tensor(-0.2...","[[tensor(0.0750), tensor(-0.0281), tensor(-0.0...","[[tensor(0.6421), tensor(0.2579), tensor(0.139...","[[tensor(0.0608), tensor(0.0244), tensor(0.013...","[[tensor(0.2878), tensor(0.0839), tensor(-0.20...","[[tensor(0.0248), tensor(0.0072), tensor(-0.01...",[[tensor(0.8521)]],[[0.8521486]],[[tensor(0.6136)]],[[0.6136041]]
4,"[[tensor(-0.2659), tensor(-0.5713), tensor(-0....","[[tensor(-0.0230), tensor(-0.0494), tensor(-0....","[[tensor(-0.6701), tensor(-0.6523), tensor(-0....","[[tensor(-0.0528), tensor(-0.0514), tensor(-0....","[[tensor(-0.8285), tensor(-0.8042), tensor(-0....","[[tensor(-0.0671), tensor(-0.0652), tensor(-0....",[[tensor(0.9279)]],[[0.92792183]],[[tensor(0.8334)]],[[0.83341897]]


In [364]:
def concat_sembedding_fe(df,c,n):
    #0->s | 1->r | 2->h
    for i in tqdm(range(len(df[c[0]]))):
        if n==7:
            c1=torch.cat((df[c[2]][i],df[c[0]][i],df[c[1]][i]),axis=1).cpu().detach().numpy() #h;s;r ->768*3=2304
        else:
            c1=torch.cat((df[c[2]][i],df[c[1]][i]),axis=1).cpu().detach().numpy() #h;r
        c2=np.tensordot(df[c[2]][i],df[c[0]][i]).reshape(1,1) #h*s ->1
        c3=np.tensordot(df[c[2]][i],df[c[1]][i]).reshape(1,1) #h*r ->1
        c4=np.absolute(np.subtract(df[c[2]][i].cpu().detach().numpy(),df[c[0]][i].cpu().detach().numpy())) #|h-s| -> 768
        c5=np.absolute(np.subtract(df[c[2]][i].cpu().detach().numpy(),df[c[1]][i].cpu().detach().numpy())) #|h-r| -> 768
        dfTemp=pd.DataFrame(np.concatenate((c1,c2,c3,c4,c5),axis=1)) #3074-3842
        dfNew=dfTemp.copy() if i==0 else pd.concat([dfNew,dfTemp],axis=0)
    return dfNew

### FE & Concat 
   - (h ; r ; hxs ; hxr ; |h-s| , |h-r|) -> Comet HTER
   - (h ; s ; r ; hxs ; hxr ; |h-s| , |h-r|) -> Comet RANK

In [379]:
df_embd_concat6,df_embd_concat7={},{}
for i in ['cs_en','de_en','ru_en','zh_en','en_fi','en_zh']:
    for j in [6,7]:
        df_Temp=concat_sembedding_fe(df_embd[f'{i}_se'],['se_source','se_reference','se_translation'],j).copy()
        df_Temp.insert(0,'z-score',df_metr[f'{i}_metrics']['z-score'].tolist())
        df_Temp.insert(1,'avg-score',df_metr[f'{i}_metrics']['avg-score'].tolist())
        df_embd_concat6.update({i:df_Temp}) if j==6 else df_embd_concat7.update({i:df_Temp})

  0%|          | 0/11585 [00:00<?, ?it/s]

  0%|          | 0/11585 [00:00<?, ?it/s]

  0%|          | 0/21704 [00:00<?, ?it/s]

  0%|          | 0/21704 [00:00<?, ?it/s]

  0%|          | 0/17980 [00:00<?, ?it/s]

...

  0%|          | 0/26419 [00:00<?, ?it/s]

  0%|          | 0/6748 [00:00<?, ?it/s]

  0%|          | 0/6748 [00:00<?, ?it/s]

  0%|          | 0/10221 [00:00<?, ?it/s]

  0%|          | 0/10221 [00:00<?, ?it/s]

In [383]:
df_embd_concat6['de_en'].head()

,z-score,avg-score,0,1,2,3,4,5,6,7,...,3064,3065,3066,3067,3068,3069,3070,3071,3072,3073
0,-0.345024,76.0,0.035101,0.171241,-0.221263,0.020823,-0.116876,0.033169,0.342028,0.020626,...,0.136864,0.340370,0.005815,0.022301,0.159110,0.239630,0.171028,0.075297,0.238582,0.073580
0,0.903800,97.5,0.441910,0.401939,-0.287938,0.002537,-0.386787,0.280978,0.178812,0.352004,...,0.088061,0.075753,0.147820,0.009613,0.068643,0.028125,0.103776,0.009230,0.075397,0.045217
0,0.700503,94.0,0.181786,0.049209,0.004193,-0.190124,-0.288409,-0.148309,-0.282938,-0.236949,...,0.102704,0.036216,0.119255,0.154581,0.218832,0.251640,0.540520,0.081000,0.079354,0.290727
0,-1.256572,51.5,0.287785,0.083903,-0.206311,-0.353662,0.090711,0.452876,0.530199,-0.555253,...,0.037117,0.256305,0.035243,0.143557,0.233543,0.335333,0.008892,0.103740,0.253465,0.075736
0,0.293909,87.0,-0.828512,-0.804248,-0.777526,-0.935924,0.383952,-0.719012,-0.317993,0.263098,...,0.027745,0.134219,0.113438,0.239927,0.019111,0.071061,0.149642,0.166202,0.157245,0.178329


In [122]:
#Activation functions
from torch import nn

def build_activation(self, activation: str) -> nn.Module:
    if hasattr(nn, activation):
        return getattr(nn, activation)()
    elif activation == 'Swish':
        return Swish()
    else:
        raise Exception('{} invalid activation function.'.format(activation))

def swish(input: torch.Tensor) -> torch.Tensor:
    return input * torch.sigmoid(input)

class Swish(nn.Module):
    def __init__(self) -> None:
        super().__init__()  #init the base class

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return swish(input)

In [124]:
#Build activation
class FeedForward(nn.Module):
    def __init__(
        self,
        in_dim: int,
        out_dim: int = 1,
        hidden_sizes: str = '3072,1536,768',
        activations: str = 'Sigmoid',
        final_activation: str = 'Sigmoid',
        dropout: float = 0.0,
    ) -> None:
        super().__init__()

        if isinstance(hidden_sizes, str):
            hidden_sizes = [int(x) for x in hidden_sizes.split(',')]

        if isinstance(hidden_sizes, int):
            hidden_sizes = [hidden_sizes]

        activation_func = build_activation(activations)

        self.ff = torch.nn.Sequential()
        self.ff.add_module('linear_1', nn.Linear(in_dim, hidden_sizes[0]))
        self.ff.add_module('activation_1', activation_func)
        self.ff.add_module('dropout_1', nn.Dropout(dropout))

        for i in range(1, len(hidden_sizes)):
            self.ff.add_module(
                'linear_{}'.format(i + 1),
                nn.Linear(int(hidden_sizes[i - 1]), hidden_sizes[i]),
            )
            self.ff.add_module('activation_{}'.format(i + 1), activation_func)
            self.ff.add_module('dropout_{}'.format(i + 1), nn.Dropout(dropout))

        self.ff.add_module(
            'linear_{}'.format(len(hidden_sizes) + 1),
            nn.Linear(hidden_sizes[-1], int(out_dim)),
        )
        if final_activation:
            final_activation = build_activation(final_activation)
            self.ff.add_module(
                'activation_{}'.format(len(hidden_sizes) + 1), final_activation
            )

    def forward(self, in_features: torch.Tensor) -> torch.Tensor:
        return self.ff(in_features)

In [390]:
from sklearn.preprocessing import StandardScaler

def make_model(df,max_depth):
    
    X = df.iloc[:,2:].values
    y = df['z-score'].values
    
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6)

    display(dfp.corr()[['avg-score','z-score']])
    
    train_dmatrix = xg.DMatrix(data = X_train, label = y_train)
    test_dmatrix = xg.DMatrix(data = X_test, label = y_test)

    param = {'booster':'gbtree', 'objective':'reg:squarederror', 'learning_rate' : 0.1,
             'max_depth' : max_depth, 'min_child_weight' : 1}

    xgb_r = xg.train(params = param, dtrain = train_dmatrix, num_boost_round = 10)
    y_pred = xgb_r.predict(test_dmatrix)

    print('Pearson:',scipy.stats.pearsonr(y_test, y_pred)[0])
    print('KT:',scipy.stats.kendalltau(y_test, y_pred)[0])

    w = 800
    fig = px.scatter(x = y_test, y = y_pred, trendline='ols', width = w, height = w/1.5)
    fig.update_traces(marker_size=1.5)
    fig.show()

In [394]:
#make_model(df_embd_concat6['de_en'],4)
#make_model(df_embd_concat7['de_en'],4)
make_model(de_en_concat,4)

,avg-score,z-score
avg-score,1.000000,0.891217
z-score,0.891217,1.000000
cosine_similarity,0.256674,0.289592
jaccard_similarity,0.258798,0.289865
dice_similarity,0.241674,0.273022
bleu1,0.264835,0.299173
bleu2,0.240299,0.270266
bleu3,0.219576,0.244203
bleu4,0.190248,0.209855
difflib_ratio,0.257643,0.292754


Pearson: 0.31925961390313157
KT: 0.2122402617152303


In [506]:
def model_predict_mlp(df,n_iter):
    X = df.iloc[:,2:].values.copy()
    y = df['z-score'].values.copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6)
    #hidden=(1536,768,384) if size==3074 else =(1920,960,480)
    
    regr = MLPRegressor(hidden_layer_sizes=(3072,1536,768), 
                        activation='tanh',
                        learning_rate='adaptive',
                        learning_rate_init=3e-05,
                        batch_size=16, 
                        random_state=6, 
                        max_iter=int(n_iter),
                        solver='adam',
                        verbose=True).fit(X_train, y_train)
    y_pred=regr.predict(X_test)
    print('Pearson:',scipy.stats.pearsonr(y_test, y_pred)[0])
    print('KT:',scipy.stats.kendalltau(y_test, y_pred)[0])
    return regr

### 1st : 30 epochs

In [193]:
x_col=de_en_concat.columns[2:]
y_col=de_en_concat.columns[1]#z-score
regr30=model_predict_mlp(de_en_concat,x_col,y_col,30)

Iteration 1, loss = 0.37071385
Iteration 2, loss = 0.33685365
Iteration 3, loss = 0.32717588
Iteration 4, loss = 0.32224803
Iteration 5, loss = 0.31466602
Iteration 6, loss = 0.30786935
Iteration 7, loss = 0.30110355
Iteration 8, loss = 0.29691144
Iteration 9, loss = 0.29031816
Iteration 10, loss = 0.28303166
Iteration 11, loss = 0.27700591
Iteration 12, loss = 0.27248520
Iteration 13, loss = 0.26520354
Iteration 14, loss = 0.26104255
Iteration 15, loss = 0.25371665
Iteration 16, loss = 0.24810561
Iteration 17, loss = 0.24264447
Iteration 18, loss = 0.23959404
Iteration 19, loss = 0.23299636
Iteration 20, loss = 0.22781515
Iteration 21, loss = 0.22417806
Iteration 22, loss = 0.21687773
Iteration 23, loss = 0.21570776
Iteration 24, loss = 0.21033043
Iteration 25, loss = 0.20721908
Iteration 26, loss = 0.20356963
Iteration 27, loss = 0.19889821
Iteration 28, loss = 0.19338948
Iteration 29, loss = 0.19135463
Iteration 30, loss = 0.18506716


In [195]:
y_pred=regr.predict(X_test)
print('Pearson:',scipy.stats.pearsonr(y_test, y_pred)[0])
print('KT:',scipy.stats.kendalltau(y_test, y_pred)[0])

Pearson: 0.37003429600289006
KT: 0.26095476139818735


### 2nd: 60 Epochs

In [196]:
'''
regr = MLPRegressor(hidden_layer_sizes=(3072,1536,768), 
                    activation='tanh',
                    learning_rate='adaptive',
                    learning_rate_init=3e-05,
                    batch_size=16, 
                    random_state=6, 
                    max_iter=60,
                    solver='adam',
                    verbose=True).fit(X_train, y_train)
y_pred=regr.predict(X_test)
print('Pearson:',scipy.stats.pearsonr(y_test, y_pred)[0])
print('KT:',scipy.stats.kendalltau(y_test, y_pred)[0])
'''
regr60=model_predict_mlp(de_en_concat,x_col,y_col,60)

Iteration 1, loss = 0.37071385
Iteration 2, loss = 0.33685365
Iteration 3, loss = 0.32717588
Iteration 4, loss = 0.32224803
Iteration 5, loss = 0.31466602
Iteration 6, loss = 0.30786935
Iteration 7, loss = 0.30110355
Iteration 8, loss = 0.29691144
Iteration 9, loss = 0.29031816
Iteration 10, loss = 0.28303166
Iteration 11, loss = 0.27700591
Iteration 12, loss = 0.27248520
Iteration 13, loss = 0.26520354
Iteration 14, loss = 0.26104255
Iteration 15, loss = 0.25371665
Iteration 16, loss = 0.24810561
Iteration 17, loss = 0.24264447
Iteration 18, loss = 0.23959404
Iteration 19, loss = 0.23299636
Iteration 20, loss = 0.22781515
Iteration 21, loss = 0.22417806
Iteration 22, loss = 0.21687773
Iteration 23, loss = 0.21570776
Iteration 24, loss = 0.21033043
Iteration 25, loss = 0.20721908
Iteration 26, loss = 0.20356963
Iteration 27, loss = 0.19889821
Iteration 28, loss = 0.19338948
Iteration 29, loss = 0.19135463
Iteration 30, loss = 0.18506716
Iteration 31, loss = 0.18312477
Iteration 32, los

### 3rd: 5 Epochs

In [199]:
regr5=model_predict_mlp(de_en_concat,x_col,y_col,5)

Iteration 1, loss = 0.37071385
Iteration 2, loss = 0.33685365
Iteration 3, loss = 0.32717588
Iteration 4, loss = 0.32224803
Iteration 5, loss = 0.31466602
Pearson: 0.36326056784351546
KT: 0.24393965061922507


### 4th: 4 Epochs

In [200]:
regr4=model_predict_mlp(de_en_concat,x_col,y_col,4)

Iteration 1, loss = 0.37071385
Iteration 2, loss = 0.33685365
Iteration 3, loss = 0.32717588
Iteration 4, loss = 0.32224803
Pearson: 0.37312202550949825
KT: 0.2571040093291824


### 5th: 3 Epochs

In [201]:
regr3=model_predict_mlp(de_en_concat,x_col,y_col,3)

Iteration 1, loss = 0.37071385
Iteration 2, loss = 0.33685365
Iteration 3, loss = 0.32717588
Pearson: 0.363930483264006
KT: 0.2506423531972065


### 6th: 10epochs

In [ ]:
model_predict_mlp(de_en_concat,10)

In [408]:
regr10=model_predict_mlp(df_embd_concat6['de_en'],10)

Iteration 1, loss = 0.36746678
Iteration 2, loss = 0.33618903
Iteration 3, loss = 0.32832093
Iteration 4, loss = 0.32032953
Iteration 5, loss = 0.31546915
Iteration 6, loss = 0.30829017
Iteration 7, loss = 0.30342242
Iteration 8, loss = 0.29606108
Iteration 9, loss = 0.29008213
Iteration 10, loss = 0.28451411
Pearson: 0.3813061037790482
KT: 0.2646310398269926


In [399]:
regr10=model_predict_mlp(df_embd_concat7['de_en'],10)

Iteration 1, loss = 0.37527781
Iteration 2, loss = 0.33522519
Iteration 3, loss = 0.32559197
Iteration 4, loss = 0.31826566
Iteration 5, loss = 0.31313151
Iteration 6, loss = 0.30498751
Iteration 7, loss = 0.29703241
Iteration 8, loss = 0.28977231
Iteration 9, loss = 0.28126215
Iteration 10, loss = 0.27423839
Pearson: 0.3790949366727308
KT: 0.266497499545027


In [ ]:
pipe_cv_mlpc = Pipeline([('cv', CountVectorizer(ngram_range=(1, 2), stop_words=[".", "...", "!", "?"], max_df=0.8)),
                         ('mlpc', MLPClassifier(hidden_layer_sizes=(50, 50), learning_rate="adaptive", random_state=15))])

pipe_tfidf_mlpc = Pipeline([('cv', TfidfVectorizer(ngram_range=(1, 2), stop_words=[".", "...", "!", "?"])),
                            ('mlpc', MLPClassifier(hidden_layer_sizes=(50, 50), learning_rate="adaptive", random_state=15))])

grid_params_cv_mlpc = [{"cv__binary": [True, False], "mlpc__activation": ['tanh', 'relu'],'mlpc__alpha': [0.0001, 0.05]}]

grid_params_tfidf_mlpc = [{"tfidf__max_df": np.arange(0.8, 1.05, 0.05),
                           "tfidf__binary": [True, False],
                           "mlpc__activation": ['tanh', 'relu'],
                           'mlpc__alpha': [0.0001, 0.05]}]

gs_cv_mlpc = GridSearchCV(estimator=pipe_cv_mlpc,
                          param_grid=grid_params_cv_mlpc,
                          scoring=scoring,
                          cv=cv,
                          n_jobs=jobs)

In [ ]:
'''
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.layers import LSTM
#from tensorflow.keras.layers import Dropout
#from tensorflow.keras import optimizers
#from tensorflow.keras.utils import plot_model

regressor = Sequential()
regressor.add(LSTM(units = 3072,  activation='tanh', return_sequences = True, input_shape = (X_train.shape[1],1)))
#regressor.add(LSTM(units = 3072,  activation='tanh', return_sequences = True, input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units = 1536, activation='tanh', return_sequences = True))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units = 768, activation='tanh', return_sequences = True))
regressor.add(Dropout(0.1))

regressor.add(Dense(units = 1, activation='tanh'))

adam = optimizers.Adam(lr=3e-05)

regressor.compile(optimizer=adam, loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs = 50, batch_size = 16, verbose=2)
'''

### Update Metrics

In [21]:
dfp.insert(len(dfp.columns),columns[3],np.nan)
for i in range(len(dfp)): dfp.loc[i,columns[3]]=np.ravel(de_en_sm[columns[3]][i])
dfp.head()

,source,reference,translation,avg-score,z-score,cosine_similarity,jaccard_similarity,dice_similarity,ratio,de_en_ref_tra_np
0,zeitlupentempo ma en spitzbergen sechs tiere s...,timeless pace measure equipped six animal broa...,slow speed measured researcher svalbard fitted...,76.0,-0.345024,0.133631,0.035714,0.133333,0.492308,0.703085
1,sagte bereiche ruhige treffpunkte fl chtlingen...,said area offer quiet meeting point refugee vo...,said space provided calm meeting point refugee...,97.5,0.903800,0.625000,0.330579,0.625000,0.752294,0.931923
2,f r gesch ftsleute b kleiner trost f r kunden ...,businessmen b small consolation customer rotte...,small consolation business located along b roa...,94.0,0.700503,0.400892,0.145833,0.400000,0.517986,0.814447
3,f higkeit sei m glicherweise angeboren entwick...,ability may born developed gender maturity,ability may innate may develop animal reach se...,51.5,-1.256572,0.338062,0.100000,0.333333,0.653465,0.852149
4,wassertemperaturen sechs grad celsius bevorzug...,prefer water temperature around six degree cel...,generally come surface winter prefer water tem...,87.0,0.293909,0.824958,0.560000,0.823529,0.636943,0.927922


In [22]:
update_df_metrics(dfp)

,cosine_similarity,jaccard_similarity,dice_similarity,ratio,de_en_ref_tra_np
pearson,0.286883,0.289540,0.267932,0.292640,0.321612
kendall,0.200043,0.204982,0.191643,0.214853,0.225904
spearman,0.293065,0.300199,0.279770,0.315358,0.329877


In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, f1_score, make_scorer
#from mlxtend.classifier import EnsembleVoteClassifier
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler

In [86]:
# Construct some pipelines
pipe_cv_cnb = Pipeline([('cv', CountVectorizer()),
                        ('cnb', ComplementNB())])

pipe_tfidf_cnb = Pipeline([('tfidf', TfidfVectorizer()),
                           ('cnb', ComplementNB())])

pipe_cv_knn = Pipeline([('cv', CountVectorizer()),
                        ('knn', KNeighborsClassifier(metric='cosine'))])

pipe_tfidf_knn = Pipeline([('tfidf', TfidfVectorizer()),
                           ('knn', KNeighborsClassifier(metric='cosine'))])

pipe_cv_knc = Pipeline([('cv', CountVectorizer()),
                        ('knc', NearestCentroid())])

pipe_tfidf_knc = Pipeline([('tfidf', TfidfVectorizer()),
                           ('knc', NearestCentroid())])

pipe_cv_log = Pipeline([('cv', CountVectorizer()),
                        ('log', SGDClassifier(random_state=15))])

pipe_tfidf_log = Pipeline([('tfidf', TfidfVectorizer()),
                           ('log', SGDClassifier(random_state=15))])

pipe_cv_sgd = Pipeline([('cv', CountVectorizer()),
                        ('sgd', SGDClassifier(random_state=15))])

pipe_tfidf_sgd = Pipeline([('tfidf', TfidfVectorizer()),
                           ('sgd', SGDClassifier(random_state=15))])

pipe_cv_rfc = Pipeline([('cv', CountVectorizer(ngram_range=(1, 2), stop_words=[".", "...", "!", "?"], max_df=0.8)),
                        ('rfc', RandomForestClassifier(class_weight='balanced', random_state=15))])

pipe_tfidf_rfc = Pipeline([('tfidf', TfidfVectorizer()),
                           ('rfc', RandomForestClassifier(class_weight='balanced', random_state=15))])

pipe_cv_mlpc = Pipeline([('cv', CountVectorizer(ngram_range=(1, 2), stop_words=[".", "...", "!", "?"], max_df=0.8)),
                         ('mlpc', MLPClassifier(hidden_layer_sizes=(50, 50), learning_rate="adaptive",
                                                random_state=15))])

pipe_tfidf_mlpc = Pipeline([('cv', TfidfVectorizer(ngram_range=(1, 2), stop_words=[".", "...", "!", "?"])),
                            ('mlpc', MLPClassifier(hidden_layer_sizes=(50, 50), learning_rate="adaptive",
                                                   random_state=15))])

pipe_cv_pac = Pipeline([('cv', CountVectorizer()),
                        ('pac', PassiveAggressiveClassifier(class_weight='balanced', random_state=15))])

pipe_tfidf_pac = Pipeline([('tfidf', TfidfVectorizer()),
                           ('pac', PassiveAggressiveClassifier(class_weight='balanced', random_state=15))])

### Uploading Testing Data

In [409]:
test_cs_en = pd.read_csv(os.path.join(os.getcwd(),'test/cs-en/scores.csv'))
display(test_cs_en.head())
test_de_en = pd.read_csv(os.path.join(os.getcwd(),'test/de-en/scores.csv'))
display(test_de_en.head())
test_ru_en = pd.read_csv(os.path.join(os.getcwd(),'test/ru-en/scores.csv'))
display(test_ru_en.head())
test_zh_en = pd.read_csv(os.path.join(os.getcwd(),'test/zh-en/scores.csv'))
display(test_zh_en.head())
test_en_fi = pd.read_csv(os.path.join(os.getcwd(),'test/en-fi/scores.csv'))
display(test_en_fi.head())
test_en_zh = pd.read_csv(os.path.join(os.getcwd(),'test/en-zh/scores.csv'))
display(test_en_zh.head())

,source,reference,translation
0,"Památník, důstojné pietní místo, stojí vůlí dě...","The monument, a dignified piecemeal place, sta...","The memorial, a solemn place of commemoration,..."
1,Pracovník centra Čang Č-čung sdělil agentuře N...,Centre worker Zhang Zu-chung told the New Chin...,Centre worker Chang Chi-Chung told New China t...
2,Veterináři nicméně odeberou namátkové vzorky v...,"However, veterinarians take random samples of ...","However, veterinarians are taking samples of e..."
3,Uživatel @TheePharoah jí neustále retweetoval ...,User @ TheePharoah constantly retweeted her po...,A user with the handle @TheePharoah was being ...
4,Lucii bylo tehdy pouhých 19 let a rozhodně net...,Lucia was only 19 at the time and certainly ha...,"At that time, Lucie was only 19 years old, and..."


,source,reference,translation
0,Das Publikum ist fast gleichmäßig zwischen Sch...,The audience is almost evenly split between bl...,The audience is almost evenly split between bl...
1,Du kannst ihre Energie durch den Bildschirm sp...,"You can feel their energy through the screen. """"","You can feel her energy through the screen."""
2,"Da die Adresse unbekannt ist, wird die Mithilf...","As the address is unknown, the help of the pop...","As the address is unknown, the assistance of t..."
3,"Arsenal-Manager Arsene Wenger, dessen Verein i...","Arsenal manager Arsene Wenger, whose club is o...","Arsenal manager Arsene Wenger, whose club is o..."
4,Landwirtschaftsminister im Interview - Wie sch...,Agriculture Minister in the interview - How do...,Minister of Agriculture in interview – How do ...


,source,reference,translation
0,Через полчаса обуглившийся клубень достают и п...,"After half an hour, the charred tuber is taken...","After half-an-hour, the charred tuber is retri..."
1,"Здесь никто не думает отменять смертную казнь,...","Here, no one thinks to abolish the death penal...","Here, no one is concerned with abolishing the ..."
2,"Собеседники ""Известий"" в ОНФ отмечают, что док...","The interlocutors of"" Izvestiya ""in the onf no...",Izvestia’s sources in the ONF note that the re...
3,На древней Венере могли существовать океаны.,On the ancient Venus could exist in the oceans.,Oceans could have existed on ancient Venus.
4,До этого момента убийства оставались лишь исто...,"Up to this point, the murders were just a stor...","Up until this point, the murders have remained..."


,source,reference,translation
0,已经批准筹建的，暂停批准开业,"Where the preparation has been approved, the a...",Approval of opening on these establishments wi...
1,王丰源在首发式发言中说，来美国前想找本书看看别人的经验，但他翻遍新华书店没找到关于留学美国中...,"In his opening speech, Mr. Wang said he wanted...",Wang Fengyuan spoke at the launch of his new b...
2,“如果你不致力于创造透明文化，你会失去人才，”维特拉诺说道。,"""if you're not committed to creating a culture...","""If you're not committed to creating a culture..."
3,不过前提是多国联军先停止对也门的袭击。,"The premise, however, is that the coalition fo...","However, the premise is that the multinational..."
4,“在此之前，我和前男友住在骑士桥的一个更大的房子里，”乔安妮说道。,"""before that, my ex and I lived in a bigger ho...","""Before this, I was living with my ex in Knigh..."


,source,reference,translation
0,One local resident who did not wish to be name...,"Eräs paikallinen asukas, joka ei halunnut nime...",Toisen nimettömänä pysyttelevän asukkaan mukaa...
1,"Still, she clings to a chant she's committed t...",Silti hän takertuu chant hän on sitoutunut mui...,"Silti hän luottaa edelleen iskulauseeseen, jon..."
2,"I don't want to be asked, 'What were you doing...","En halua, että minulta kysytään: ""Mitä te teit...","En halua, että kenenkään tarvitsee kysyä minul..."
3,"""I wouldn't say it was a lie – that's a pretty...","""En sanoisi, että se oli valhe - se on aika ro...","En sanoisi, että se oli valhe, se on aika kova..."
4,Kari Kola took part in the opening ceremony of...,Kari Kola osallistui valon vuoden avajaisiin v...,Kari Kola oli mukana Valon teemavuoden avajais...


,source,reference,translation
0,The future and the destinies of the citizens o...,世界上每个国家公民的未来和命运日益联系在一起。,世界各国人民前途命运越来越紧密地联系在一起。
1,"After all that hard work, the finished result ...",经过那么多的努力，最终的结果现在已经可以揭晓了。,经过这么艰辛的工作，最终的结果现在才得以公布。
2,Author: researcher of Suning Institute of Fina...,作者：苏宁金融研究所研究员，财经专栏作家，财经评论员。,作者：苏宁金融研究院特约研究员，财经专栏作家，财经评论员。
3,“The Great Wall” tells the story of a Chinese ...,《长城》讲述了古代一支中国精锐部队在世界著名的中国长城上与怪物桃蒂英勇作战的故事。,《长城》讲述了在古代，一支中国精英部队为保卫人类，在举世闻名的长城上与怪兽饕餮进行生死决战的故事。
4,Our comrades from the Political Bureau should ...,政治局同志要学习历史，讲道理，不能混淆公、私利益，叫白黑，模糊义与利的界限，处理基于裙带关系...,中央政治局的同志都应该明史知理，不能颠倒了公私、混淆了是非、模糊了义利、放纵了亲情，要带头树...


### Cleaning Testing Data

In [457]:
dft_cs_en = pd.DataFrame()
dft_cs_en['source'] = preprocessing(test_cs_en, 'source',lng[2])
dft_cs_en['reference'] = preprocessing(test_cs_en, 'reference',lng[1])
dft_cs_en['translation'] = preprocessing(test_cs_en, 'translation',lng[1])
display(dft_cs_en.head())

dft_de_en = pd.DataFrame()
dft_de_en['source'] = preprocessing(test_de_en, 'source',lng[0])
dft_de_en['reference'] = preprocessing(test_de_en, 'reference',lng[1])
dft_de_en['translation'] = preprocessing(test_de_en, 'translation',lng[1])
display(dft_de_en.head())

dft_ru_en = pd.DataFrame()
dft_ru_en['source'] = preprocessing_russian(test_ru_en, 'source',lng[3])
dft_ru_en['reference'] = preprocessing(test_ru_en, 'reference',lng[1])
dft_ru_en['translation'] = preprocessing(test_ru_en, 'translation',lng[1])
display(dft_ru_en.head())

dft_zh_en = pd.DataFrame()
dft_zh_en['source'] = preprocessing_chinese(test_zh_en, 'source',lng[5])
dft_zh_en['reference'] = preprocessing(test_zh_en, 'reference',lng[1])
dft_zh_en['translation'] = preprocessing(test_zh_en, 'translation',lng[1])
display(dft_zh_en.head())

dft_en_fi = pd.DataFrame()
dft_en_fi['source'] = preprocessing(test_en_fi, 'source',lng[1])
dft_en_fi['reference'] = preprocessing(test_en_fi, 'reference',lng[4])
dft_en_fi['translation'] = preprocessing(test_en_fi, 'translation',lng[4])
display(dft_en_fi.head())

dft_en_zh = pd.DataFrame()
dft_en_zh['source'] = preprocessing(test_en_zh, 'source',lng[1])
dft_en_zh['reference'] = preprocessing_chinese(test_en_zh, 'reference',lng[5])
dft_en_zh['translation'] = preprocessing_chinese(test_en_zh, 'translation',lng[5])
display(dft_en_zh.head())

  0%|          | 0/8732 [00:00<?, ?it/s]

  0%|          | 0/8732 [00:00<?, ?it/s]

  0%|          | 0/8732 [00:00<?, ?it/s]

,source,reference,translation
0,pamatnik dustojne pietni misto stoji vuli deji...,monument dignified piecemeal place stand histo...,memorial solemn place commemoration stand than...
1,pracovnik centrum cang ccung sdelil agenture n...,centre worker zhang zuchung told new china new...,centre worker chang chichung told new china pa...
2,veterinari nicmene odeberou namatkove vzorky v...,however veterinarian take random sample egg ch...,however veterinarian taking sample egg chicken...
3,uzivatel theepharoah ji neustale retweetoval p...,user theepharoah constantly retweeted post wall,user handle theepharoah constantly retweeted o...
4,lucii tehdy pouhych let a rozhodne netouzila t...,lucia time certainly desire hang career nail,time lucie year old definitely want put brake ...


  0%|          | 0/28404 [00:00<?, ?it/s]

...

,source,reference,translation
0,one local resident wish named said ridiculous ...,era paikallinen asukas halunnut nimeaan sanoi ...,toisen nimettomana pysyttelevan asukkaan naure...
1,still cling chant shes committed memory since ...,silti han takertuu chant han sitoutunut muisti...,silti han luottaa edelleen iskulauseeseen han ...
2,dont want asked face much death didnt say going,halua etta kysytaan mita teitte suuren kuolema...,halua etta kenenkaan tarvitsee kysya missa taa...
3,wouldnt say lie thats pretty bold accusation,sanoisi etta valhe aika rohkea syytos,sanoisi etta valhe aika kova syytos
4,kari kola took part opening ceremony year ligh...,kari kola osallistui valon vuoden avajaisiin v...,kari kola mukana valon teemavuoden avajaisissa...


  0%|          | 0/22128 [00:00<?, ?it/s]

  0%|          | 0/22128 [00:00<?, ?it/s]

  0%|          | 0/22128 [00:00<?, ?it/s]

,source,reference,translation
0,future destiny citizen every country world inc...,世界 上 每个 国家 公民 的 未来 和 命运 日益 联系 在 一起,世界 各国 人民 前途 命运 越来越 紧密 地 联系 在 一起
1,hard work finished result ready unveiled,经过 那么 多 的 努力 最终 的 结果 现在 已经 可以 揭晓 了,经过 这么 艰辛 的 工作 最终 的 结果 现在 才 得以 公布
2,author researcher suning institute finance fin...,作者 苏宁 金融 研究所 研究员 财经 专栏作家 财经 评论员,作者 苏宁 金融 研究院 特约 研究员 财经 专栏作家 财经 评论员
3,great wall tell story chinese elite troop anci...,长城 讲述 了 古代 一支 中国 精锐部队 在 世界 著名 的 中国 长城 上 与 怪物 桃...,长城 讲述 了 在 古代 一支 中国 精英 部队 为 保卫 人类 在 举世闻名 的 长城 上...
4,comrade political bureau learn history reasona...,政治局 同志 要 学习 历史 讲道理 不能 混淆 公 私 利益 叫白 黑 模糊 义与利 的 ...,中央政治局 的 同志 都 应该 明史 知理 不能 颠倒 了 公私 混淆 了 是非 模糊 了 ...


### Metrics Testing Data

In [503]:
dft_metr={}
dft_metr.update({'cs_en_metrics':get_metrics(dft_cs_en,dft_cs_en.columns[:3])})
display(dft_metr['cs_en_metrics'].head())
dft_metr.update({'de_en_metrics':get_metrics(dft_de_en,dft_de_en.columns[:3])})
display(dft_metr['de_en_metrics'].head())
dft_metr.update({'ru_en_metrics':get_metrics(dft_ru_en,dft_ru_en.columns[:3])})
display(dft_metr['ru_en_metrics'].head())
dft_metr.update({'zh_en_metrics':get_metrics(dft_zh_en,dft_zh_en.columns[:3])})
display(dft_metr['zh_en_metrics'].head())
dft_metr.update({'en_fi_metrics':get_metrics(dft_en_fi,dft_en_fi.columns[:3])})
display(dft_metr['en_fi_metrics'].head())
dft_metr.update({'en_zh_metrics':get_metrics(dft_en_zh,dft_en_zh.columns[:3])})
display(dft_metr['en_zh_metrics'].head())

  0%|          | 0/8732 [00:00<?, ?it/s]

,source,reference,translation,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,pamatnik dustojne pietni misto stoji vuli deji...,monument dignified piecemeal place stand histo...,memorial solemn place commemoration stand than...,0.445132,0.173469,0.444444,0.421053,2.162952e-01,1.429167e-01,2.797353e-78,0.630037,67,69.0,0.459516,0.157143,0.481686
1,pracovnik centrum cang ccung sdelil agenture n...,centre worker zhang zuchung told new china new...,centre worker chang chichung told new china pa...,0.544949,0.290859,0.538462,0.442364,3.037302e-01,1.949160e-01,3.240088e-78,0.741176,72,34.0,0.487549,0.203704,0.434722
2,veterinari nicmene odeberou namatkove vzorky v...,however veterinarian take random sample egg ch...,however veterinarian taking sample egg chicken...,0.700877,0.434911,0.700000,0.663222,5.463325e-01,4.591137e-01,3.821919e-01,0.803797,84,47.0,0.695148,0.423077,0.748948
3,uzivatel theepharoah ji neustale retweetoval p...,user theepharoah constantly retweeted post wall,user handle theepharoah constantly retweeted o...,0.617213,0.296296,0.615385,0.571429,4.364358e-01,3.401632e-01,5.395774e-78,0.800000,88,17.0,0.737525,0.318182,0.614754
4,lucii tehdy pouhych let a rozhodne netouzila t...,lucia time certainly desire hang career nail,time lucie year old definitely want put brake ...,0.251976,0.071429,0.250000,0.222222,7.031791e-155,5.416106e-204,1.250850e-231,0.458333,56,37.0,0.279206,0.066667,0.208333


  0%|          | 0/28404 [00:00<?, ?it/s]

,source,reference,translation,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,publikum fast gleichmaig schwarz wei aufgeteilt,audience almost evenly split black white,audience almost evenly split black white,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,100,0.0,1.000000,1.000000,0.997685
1,kannst energie bildschirm spuren,feel energy screen,feel energy screen,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.221339e-77,1.000000,100,0.0,1.000000,1.000000,0.981481
2,adresse unbekannt mithilfe bevolkerung erbeten...,address unknown help population requested hoped,address unknown assistance population requeste...,0.833333,0.612245,0.833333,0.833333,0.707107,5.034778e-01,7.262123e-78,0.880000,94,10.0,0.824713,0.500000,0.997685
3,arsenalmanager arsene wenger verein besitz ame...,arsenal manager arsene wenger whose club owned...,arsenal manager arsene wenger whose club owned...,1.000000,1.000000,1.117647,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,100,0.0,1.000000,1.000000,0.999898
4,landwirtschaftsminister interview schutzen gif...,agriculture minister interview protect poison ...,minister agriculture interview protect poisone...,0.750000,0.480000,0.750000,0.750000,0.462910,1.794120e-102,1.014894e-154,0.780488,93,22.0,0.774735,0.307692,0.877930


  0%|          | 0/13157 [00:00<?, ?it/s]

...

,source,reference,translation,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,已经 批准 筹建 的 暂停 批准 开业,preparation approved approval opening business...,approval opening establishment suspended,0.566947,0.328125,0.545455,0.354275,2.361833e-01,8.917087e-103,4.982374e-155,0.607843,79,33.0,0.406591,0.181818,0.381426
1,王丰源 在 首发式 发言 中说 来 美国 前想 找 本书 看看 别人 的 经验 但 他 翻遍...,opening speech mr wang said wanted find book p...,wang fengyuan spoke launch new book saying com...,0.643679,0.176190,0.800000,0.461538,3.328201e-01,2.435783e-01,1.566392e-01,0.578231,78,82.0,0.590657,0.224490,0.571066
2,如果 你 不 致力于 创造 透明 文化 你 会 失去 人才 维特 拉诺 说道,youre committed creating culture transparency ...,youre committed creating culture transparency ...,0.832050,0.520661,0.900000,0.727273,6.030227e-01,4.983910e-01,4.172261e-01,0.868421,95,18.0,0.813397,0.473684,0.815217
3,不过 前提 是 多国联军 先 停止 对 也门 的 袭击,premise however coalition force stop attack ye...,however premise multinational coalition stop a...,0.875000,0.691358,0.875000,0.875000,6.123724e-01,5.034778e-01,3.976354e-01,0.728814,94,27.0,0.711675,0.500000,0.793367
4,在此之前 我 和 前男友 住 在 骑士 桥 的 一个 更大 的 房子 里 乔 安妮 说道,ex lived bigger house knightsbridge joanne said,living ex knightsbridge much bigger space joan...,0.534522,0.231405,0.533333,0.500000,1.054769e-154,7.077949e-204,1.531972e-231,0.565657,78,32.0,0.575048,0.153846,0.352113


  0%|          | 0/8097 [00:00<?, ?it/s]

,source,reference,translation,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,one local resident wish named said ridiculous ...,era paikallinen asukas halunnut nimeaan sanoi ...,toisen nimettomana pysyttelevan asukkaan naure...,0.122628,0.039683,0.129032,0.115303,4.550510e-155,3.778347e-204,9.040047e-232,0.387833,58,98.0,0.285637,0.032258,0.059880
1,still cling chant shes committed memory since ...,silti han takertuu chant han sitoutunut muisti...,silti han luottaa edelleen iskulauseeseen han ...,0.626224,0.280702,0.714286,0.527633,3.883275e-01,2.304062e-01,3.869722e-78,0.660099,70,47.0,0.495758,0.240741,0.506757
2,dont want asked face much death didnt say going,halua etta kysytaan mita teitte suuren kuolema...,halua etta kenenkaan tarvitsee kysya missa taa...,0.320256,0.049861,0.347826,0.250000,1.507557e-01,8.556266e-103,5.791740e-155,0.584795,55,51.0,0.429156,0.095238,0.230230
3,wouldnt say lie thats pretty bold accusation,sanoisi etta valhe aika rohkea syytos,sanoisi etta valhe aika kova syytos,0.833333,0.612245,0.833333,0.833333,7.071068e-01,6.328783e-01,5.372850e-01,0.916667,92,4.0,0.757066,0.611111,0.806667
4,kari kola took part opening ceremony year ligh...,kari kola osallistui valon vuoden avajaisiin v...,kari kola mukana valon teemavuoden avajaisissa...,0.564076,0.237654,0.560000,0.500000,2.773501e-01,1.279447e-102,7.855727e-155,0.708333,76,49.0,0.658605,0.180000,0.506592


  0%|          | 0/22128 [00:00<?, ?it/s]

,source,reference,translation,cossim,jacc,dice,bleu1,bleu2,bleu3,bleu4,difflib,fz,med,chrf,gleu,meteor
0,future destiny citizen every country world inc...,世界 上 每个 国家 公民 的 未来 和 命运 日益 联系 在 一起,世界 各国 人民 前途 命运 越来越 紧密 地 联系 在 一起,0.444444,0.183673,0.444444,0.378979,2.513860e-01,1.830112e-01,3.228232e-78,0.553846,65,17.0,0.191244,0.173913,0.348438
1,hard work finished result ready unveiled,经过 那么 多 的 努力 最终 的 结果 现在 已经 可以 揭晓 了,经过 这么 艰辛 的 工作 最终 的 结果 现在 才 得以 公布,0.444444,0.183673,0.444444,0.460022,3.397489e-01,2.802846e-01,2.158640e-01,0.696970,68,12.0,0.271952,0.260870,0.436047
2,author researcher suning institute finance fin...,作者 苏宁 金融 研究所 研究员 财经 专栏作家 财经 评论员,作者 苏宁 金融 研究院 特约 研究员 财经 专栏作家 财经 评论员,0.870388,0.566667,1.052632,0.800000,7.302967e-01,6.465017e-01,5.253820e-01,0.923077,92,4.0,0.813137,0.588235,0.872253
3,great wall tell story chinese elite troop anci...,长城 讲述 了 古代 一支 中国 精锐部队 在 世界 著名 的 中国 长城 上 与 怪物 桃...,长城 讲述 了 在 古代 一支 中国 精英 部队 为 保卫 人类 在 举世闻名 的 长城 上...,0.540738,0.132035,0.645161,0.520000,3.894440e-01,2.740145e-01,4.580438e-78,0.704000,71,26.0,0.340457,0.244681,0.584644
4,comrade political bureau learn history reasona...,政治局 同志 要 学习 历史 讲道理 不能 混淆 公 私 利益 叫白 黑 模糊 义与利 的 ...,中央政治局 的 同志 都 应该 明史 知理 不能 颠倒 了 公私 混淆 了 是非 模糊 了 ...,0.226134,0.085714,0.240000,0.222245,6.052635e-155,4.430054e-204,9.988500e-232,0.490196,49,69.0,0.110262,0.060000,0.118110


### LaBSE Embeddings Testing Data

In [504]:
dft_embd={}
for i in ['cs_en','de_en','ru_en','zh_en','en_fi','en_zh']:
    dft_embd[f'{i}_se']=get_sentence_embedding(dft_metr[f'{i}_metrics'],dft_metr[f'{i}_metrics'].columns[:3])

  0%|          | 0/8732 [00:00<?, ?it/s]

  0%|          | 0/28404 [00:00<?, ?it/s]

  0%|          | 0/13157 [00:00<?, ?it/s]

  0%|          | 0/25352 [00:00<?, ?it/s]

  0%|          | 0/8097 [00:00<?, ?it/s]

  0%|          | 0/22128 [00:00<?, ?it/s]

### LaBSE Embeddings FE & Concatenation

In [505]:
dft_embd_concat6,dft_embd_concat7={},{}
for i in ['cs_en','de_en','ru_en','zh_en','en_fi','en_zh']:
    for j in [6,7]:
        df_Temp=concat_sembedding_fe(dft_embd[f'{i}_se'],['se_source','se_reference','se_translation'],j).copy()
        if j==6:            
            dft_embd_concat6.update({i:df_Temp})
            dft_embd_concat6[i].to_csv(f'{i}_test6.csv')
        else:
            dft_embd_concat7.update({i:df_Temp})
            dft_embd_concat7[i].to_csv(f'{i}_test7.csv')

  0%|          | 0/8732 [00:00<?, ?it/s]

  0%|          | 0/8732 [00:00<?, ?it/s]

  0%|          | 0/28404 [00:00<?, ?it/s]

  0%|          | 0/28404 [00:00<?, ?it/s]

  0%|          | 0/13157 [00:00<?, ?it/s]

...

  0%|          | 0/25352 [00:00<?, ?it/s]

  0%|          | 0/8097 [00:00<?, ?it/s]

  0%|          | 0/8097 [00:00<?, ?it/s]

  0%|          | 0/22128 [00:00<?, ?it/s]

  0%|          | 0/22128 [00:00<?, ?it/s]

### Predicting on Training Data

### cs - en (f6)

In [508]:
model_predict_mlp(df_embd_concat6['cs_en'],10)

Iteration 1, loss = 0.35821877
Iteration 2, loss = 0.31081145
Iteration 3, loss = 0.29351919
Iteration 4, loss = 0.27741928
Iteration 5, loss = 0.27061037
Iteration 6, loss = 0.25666925
Iteration 7, loss = 0.24782831
Iteration 8, loss = 0.23808703
Iteration 9, loss = 0.22891586
Iteration 10, loss = 0.21763498
Pearson: 0.5271304047601822
KT: 0.3713828861484922


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

### cs - en (f7)

In [515]:
model_predict_mlp(df_embd_concat7['cs_en'],10)

Iteration 1, loss = 0.37063329
Iteration 2, loss = 0.30520424
Iteration 3, loss = 0.28533589
Iteration 4, loss = 0.27113412
Iteration 5, loss = 0.25762515
Iteration 6, loss = 0.24794890
Iteration 7, loss = 0.23837380
Iteration 8, loss = 0.22239528
Iteration 9, loss = 0.21268400
Iteration 10, loss = 0.20210504
Pearson: 0.5328098444130042
KT: 0.37388382143637305


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

In [533]:
model_predict_mlp(df_embd_concat7['cs_en'],3)

Pearson: 0.4100216089056624
KT: 0.24851782611559436


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=3, random_state=6,
             verbose=True)

In [534]:
model_predict_mlp(df_embd_concat7['cs_en'],4)

Pearson: 0.19141975070065437
KT: 0.09638276330428744


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=4, random_state=6,
             verbose=True)

### de - en (f6)

In [509]:
model_predict_mlp(df_embd_concat6['de_en'],10)

Iteration 1, loss = 0.36746678
Iteration 2, loss = 0.33618903
Iteration 3, loss = 0.32832093
Iteration 4, loss = 0.32032953
Iteration 5, loss = 0.31546915
Iteration 6, loss = 0.30829017
Iteration 7, loss = 0.30342242
Iteration 8, loss = 0.29606108
Iteration 9, loss = 0.29008213
Iteration 10, loss = 0.28451411
Pearson: 0.3813061037790482
KT: 0.2646310398269926


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

### de - en (f7)

In [516]:
model_predict_mlp(df_embd_concat7['de_en'],10)

Iteration 1, loss = 0.37527781
Iteration 2, loss = 0.33522519
Iteration 3, loss = 0.32559197
Iteration 4, loss = 0.31826566
Iteration 5, loss = 0.31313151
Iteration 6, loss = 0.30498751
Iteration 7, loss = 0.29703241
Iteration 8, loss = 0.28977231
Iteration 9, loss = 0.28126215
Iteration 10, loss = 0.27423839
Pearson: 0.3790949366727308
KT: 0.266497499545027


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

### ru - en (f6)

In [510]:
model_predict_mlp(df_embd_concat6['ru_en'],10)

Iteration 1, loss = 0.39727002
Iteration 2, loss = 0.36719225
Iteration 3, loss = 0.34940612
Iteration 4, loss = 0.34786717
Iteration 5, loss = 0.33602334
Iteration 6, loss = 0.32852151
Iteration 7, loss = 0.32236786
Iteration 8, loss = 0.31233820
Iteration 9, loss = 0.30674591
Iteration 10, loss = 0.29915206
Pearson: 0.4013971951734677
KT: 0.2682087549228456


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

### ru - en (f7)

In [517]:
model_predict_mlp(df_embd_concat7['ru_en'],10)

Iteration 1, loss = 0.39561934
Iteration 2, loss = 0.35857720
Iteration 3, loss = 0.34895910
Iteration 4, loss = 0.33836791
Iteration 5, loss = 0.33232478
Iteration 6, loss = 0.32241036
Iteration 7, loss = 0.31183268
Iteration 8, loss = 0.30281950
Iteration 9, loss = 0.29503855
Iteration 10, loss = 0.28678209
Pearson: 0.3880180888435549
KT: 0.2680959080640839


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

### zh - en (f6)

In [511]:
model_predict_mlp(df_embd_concat6['zh_en'],10)

Iteration 1, loss = 0.37500279
Iteration 2, loss = 0.34577995
Iteration 3, loss = 0.33687770
Iteration 4, loss = 0.33061737
Iteration 5, loss = 0.32683078
Iteration 6, loss = 0.31927117
Iteration 7, loss = 0.31533776
Iteration 8, loss = 0.30773396
Iteration 9, loss = 0.30282245
Iteration 10, loss = 0.29793426
Pearson: 0.41463933755951043
KT: 0.29158547837625187


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

### zh - en (f7)

In [519]:
model_predict_mlp(df_embd_concat7['zh_en'],10)

Iteration 1, loss = 0.37350318
Iteration 2, loss = 0.34135196
Iteration 3, loss = 0.33639084
Iteration 4, loss = 0.32890800
Iteration 5, loss = 0.32168071
Iteration 6, loss = 0.31557747
Iteration 7, loss = 0.31145516
Iteration 8, loss = 0.30512497
Iteration 9, loss = 0.29816650
Iteration 10, loss = 0.29263016
Pearson: 0.42010319024236176
KT: 0.2928646535453922


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

In [527]:
model_predict_mlp(df_embd_concat7['zh_en'],3)

Iteration 1, loss = 0.37350318
Iteration 2, loss = 0.34135196
Iteration 3, loss = 0.33639084
Pearson: 0.43631751631555976
KT: 0.2966963005106453


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=3, random_state=6,
             verbose=True)

In [528]:
model_predict_mlp(df_embd_concat7['zh_en'],4)

Iteration 1, loss = 0.37350318
Iteration 2, loss = 0.34135196
Iteration 3, loss = 0.33639084
Iteration 4, loss = 0.32890800
Pearson: 0.43953208562744905
KT: 0.29877370945689075


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=4, random_state=6,
             verbose=True)

### en - fi (f6)

In [512]:
model_predict_mlp(df_embd_concat6['en_fi'],10)

Iteration 1, loss = 0.32965796
Iteration 2, loss = 0.27502482
Iteration 3, loss = 0.24222822
Iteration 4, loss = 0.23838408
Iteration 5, loss = 0.22882413
Iteration 6, loss = 0.21135357
Iteration 7, loss = 0.20462610
Iteration 8, loss = 0.19433661
Iteration 9, loss = 0.18292262
Iteration 10, loss = 0.17575556
Pearson: 0.616168459247628
KT: 0.4077756714009575


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

### en - fi (f7)

In [520]:
model_predict_mlp(df_embd_concat7['en_fi'],10)

Iteration 1, loss = 0.33087075
Iteration 2, loss = 0.25190895
Iteration 3, loss = 0.23656193
Iteration 4, loss = 0.22275641
Iteration 5, loss = 0.22345543
Iteration 6, loss = 0.19937132
Iteration 7, loss = 0.18859783
Iteration 8, loss = 0.18084269
Iteration 9, loss = 0.16154300
Iteration 10, loss = 0.15736552
Pearson: 0.6016249459501863
KT: 0.3929328574636632


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

In [524]:
model_predict_mlp(df_embd_concat7['en_fi'],3)

Iteration 1, loss = 0.33087075
Iteration 2, loss = 0.25190895
Iteration 3, loss = 0.23656193
Pearson: 0.627580470864206
KT: 0.40651384651006806


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=3, random_state=6,
             verbose=True)

In [525]:
model_predict_mlp(df_embd_concat7['en_fi'],4)

Iteration 1, loss = 0.33087075
Iteration 2, loss = 0.25190895
Iteration 3, loss = 0.23656193
Iteration 4, loss = 0.22275641
Pearson: 0.6096688826163956
KT: 0.3949951739003608


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=4, random_state=6,
             verbose=True)

In [526]:
model_predict_mlp(df_embd_concat7['en_fi'],5)

Iteration 1, loss = 0.33087075
Iteration 2, loss = 0.25190895
Iteration 3, loss = 0.23656193
Iteration 4, loss = 0.22275641
Iteration 5, loss = 0.22345543
Pearson: 0.6318907080125298
KT: 0.4127877845229412


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

### en - zh (f6)

In [513]:
model_predict_mlp(df_embd_concat6['en_zh'],10)

Iteration 1, loss = 0.38335377
Iteration 2, loss = 0.33716227
Iteration 3, loss = 0.32474425
Iteration 4, loss = 0.31278385
Iteration 5, loss = 0.30851758
Iteration 6, loss = 0.29978680
Iteration 7, loss = 0.29184230
Iteration 8, loss = 0.28235802
Iteration 9, loss = 0.27470790
Iteration 10, loss = 0.27351360
Pearson: 0.5250775283402502
KT: 0.36628879153998073


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

### en-zh (f7)

In [521]:
model_predict_mlp(df_embd_concat7['en_zh'],10)

Iteration 1, loss = 0.39187817
Iteration 2, loss = 0.34293116
Iteration 3, loss = 0.32311331
Iteration 4, loss = 0.31453086
Iteration 5, loss = 0.30472444
Iteration 6, loss = 0.29669819
Iteration 7, loss = 0.28643798
Iteration 8, loss = 0.27323454
Iteration 9, loss = 0.26754906
Iteration 10, loss = 0.26098284
Pearson: 0.49491280431317697
KT: 0.3426853909998778


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

In [522]:
model_predict_mlp(df_embd_concat7['en_zh'],5)

Iteration 1, loss = 0.39187817
Iteration 2, loss = 0.34293116
Iteration 3, loss = 0.32311331
Iteration 4, loss = 0.31453086
Iteration 5, loss = 0.30472444
Pearson: 0.5339256948745887
KT: 0.3678534979596846


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

In [523]:
model_predict_mlp(df_embd_concat7['en_zh'],4)

Iteration 1, loss = 0.39187817
Iteration 2, loss = 0.34293116
Iteration 3, loss = 0.32311331
Iteration 4, loss = 0.31453086
Pearson: 0.5267440072392957
KT: 0.36485816632871176


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=4, random_state=6,
             verbose=True)

### Uploading Data Released by Mistake 

In [641]:
dft=pd.read_csv('test-scores.csv')
dft['lp'].value_counts()

zh-en    1000
fi-en    1000
ru-en    1000
en-tr    1000
et-en    1000
cs-en    1000
de-en    1000
en-cs    1000
tr-en    1000
en-et    1000
en-ru    1000
en-de    1000
en-fi    1000
en-zh    1000
Name: lp, dtype: int64

In [642]:
test={}
slang=['cs-en','de-en','ru-en','zh-en','en-fi','en-zh']
for i in slang: test.update({i:dft[dft['lp']==i].reset_index(drop=True)})#test-mistake

In [ ]:
lng=['german','english','czech','russian','finnish','chinese-zh']
test['cs-en']['src'] = preprocessing(test['cs-en'],'src',lng[2])
test['cs-en']['ref'] = preprocessing(test['cs-en'],'ref',lng[1])
test['cs-en']['mt'] = preprocessing(test['cs-en'],'mt',lng[1])
display(test['cs-en'].head())

test['de-en']['src'] = preprocessing(test['de-en'], 'src',lng[0])
test['de-en']['ref'] = preprocessing(test['de-en'], 'ref',lng[1])
test['de-en']['mt'] = preprocessing(test['de-en'], 'mt',lng[1])
display(test['de-en'].head())

test['ru-en']['src'] = preprocessing_russian(test['ru-en'], 'src',lng[3])
test['ru-en']['ref'] = preprocessing(test['ru-en'], 'ref',lng[1])
test['ru-en']['mt'] = preprocessing(test['ru-en'], 'mt',lng[1])
display(test['ru-en'].head())

test['zh-en']['src'] = preprocessing_chinese(test['zh-en'], 'src',lng[5])
test['zh-en']['ref'] = preprocessing(test['zh-en'], 'ref',lng[1])
test['zh-en']['mt'] = preprocessing(test['zh-en'], 'mt',lng[1])
display(test['zh-en'].head())

test['en-fi']['src'] = preprocessing(test['en-fi'], 'src',lng[1])
test['en-fi']['ref'] = preprocessing(test['en-fi'], 'ref',lng[4])
test['en-fi']['mt'] = preprocessing(test['en-fi'], 'mt',lng[4])
display(test['en-fi'].head())

test['en-zh']['src'] = preprocessing(test['en-zh'], 'src',lng[1])
test['en-zh']['ref'] = preprocessing_chinese(test['en-zh'], 'ref',lng[5])
test['en-zh']['mt'] = preprocessing_chinese(test['en-zh'], 'mt',lng[5])
display(test['en-zh'].head())

In [ ]:
for i in slang: test[i]=test[i][['src','ref','mt','score','raw_score']]
    
test['cs-en']=get_metrics(test['cs-en'],test['cs-en'].columns[:5]) #ref->1 #mt->2
display(test['cs-en'].head())

test['de-en']=get_metrics(test['de-en'],test['de-en'].columns[:5])
display(test['de-en'].head())

test['ru-en']=get_metrics(test['ru-en'],test['ru-en'].columns[:5])
display(test['ru-en'].head())

test['zh-en']=get_metrics(test['zh-en'],test['zh-en'].columns[:5])
display(test['zh-en'].head())

test['en-fi']=get_metrics(test['en-fi'],test['en-fi'].columns[:5])
display(test['en-fi'].head())

test['en-zh']=get_metrics(test['en-zh'],test['en-zh'].columns[:5])
display(test['en-zh'].head())

In [648]:
dfm_embd={}
for i in slang: dfm_embd[f'{i}_se']=get_sentence_embedding(test[i],test[i].columns[:3])

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [653]:
dfm_embd_concat6,dfm_embd_concat7={},{}
for i in slang:
    for j in [6,7]:
        dfTmp=concat_sembedding_fe(dfm_embd[f'{i}_se'],['se_src','se_ref','se_mt'],j).copy()
        if j==6:            
            dfm_embd_concat6.update({i:dfTmp})
            #dfm_embd_concat6[i].to_csv(f'{i}_m6.csv')
        else:
            dfm_embd_concat7.update({i:dfTmp})
            #dfm_embd_concat7[i].to_csv(f'{i}_m7.csv')

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [654]:
for i in slang:
    dfm_embd_concat6[i].reset_index(drop=True,inplace=True)
    dfm_embd_concat6[i].insert(0,'z-score',test[i]['score'])
    dfm_embd_concat6[i].insert(1,'avg-score',test[i]['raw_score'])
    dfm_embd_concat7[i].reset_index(drop=True,inplace=True)
    dfm_embd_concat7[i].insert(0,'z-score',test[i]['score'])
    dfm_embd_concat7[i].insert(1,'avg-score',test[i]['raw_score'])

### Predicting on Testing Data

### cs - en

In [660]:
def model_pred1(df,df1,name,n_iter):
    X = df.iloc[:,2:].values
    y = df['z-score'].values
    
    X_test=df1.iloc[:,2:].values
    y_test=df1['z-score']
    regr = MLPRegressor(hidden_layer_sizes=(3072,1536,768), 
                        activation='tanh',
                        learning_rate='adaptive',
                        learning_rate_init=3e-05,
                        batch_size=16, 
                        random_state=6, 
                        max_iter=int(n_iter),
                        solver='adam',
                        verbose=True)
    regr.fit(X, y)
    #save
    pickle.dump(regr,open(f'{name}.sav','wb'))
    #loaded_model = pickle.load(open(f'{name}.sav','rb'))
    y_pred=regr.predict(X_test)
    print('Pearson:',scipy.stats.pearsonr(y_test, y_pred)[0])
    print('KT:',scipy.stats.kendalltau(y_test, y_pred)[0])
    return regr

In [663]:
model_pred1(df_embd_concat6['cs_en'],dfm_embd_concat6['cs-en'],'t6-cs-en',10)

Iteration 1, loss = 0.35137911
Iteration 2, loss = 0.30300355
Iteration 3, loss = 0.28977533
Iteration 4, loss = 0.27965698
Iteration 5, loss = 0.26420568
Iteration 6, loss = 0.25534927
Iteration 7, loss = 0.25001118
Iteration 8, loss = 0.23482186
Iteration 9, loss = 0.23054210
Iteration 10, loss = 0.22543661
Pearson: 0.25984040701923244
KT: 0.1961286543975382


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=10, random_state=6,
             verbose=True)

In [664]:
model_pred1(df_embd_concat6['cs_en'],dfm_embd_concat6['cs-en'],'t6-cs-en',3)

Iteration 1, loss = 0.35137911
Iteration 2, loss = 0.30300355
Iteration 3, loss = 0.28977533
Pearson: 0.28106758959607625
KT: 0.20571641880633726


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=3, random_state=6,
             verbose=True)

In [665]:
model_pred1(df_embd_concat6['cs_en'],dfm_embd_concat6['cs-en'],'t6-cs-en',4)

Iteration 1, loss = 0.35137911
Iteration 2, loss = 0.30300355
Iteration 3, loss = 0.28977533
Iteration 4, loss = 0.27965698
Pearson: 0.2924403242183544
KT: 0.21679801618275618


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=4, random_state=6,
             verbose=True)

In [666]:
model_pred1(df_embd_concat6['cs_en'],dfm_embd_concat6['cs-en'],'t6-cs-en-5',5)

Iteration 1, loss = 0.35137911
Iteration 2, loss = 0.30300355
Iteration 3, loss = 0.28977533
Iteration 4, loss = 0.27965698
Iteration 5, loss = 0.26420568
Pearson: 0.2738158865802154
KT: 0.19891607462666275


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

In [667]:
model_pred1(df_embd_concat7['cs_en'],dfm_embd_concat7['cs-en'],'t7-cs-en-3',3)
model_pred1(df_embd_concat7['cs_en'],dfm_embd_concat7['cs-en'],'t7-cs-en-4',4)
model_pred1(df_embd_concat7['cs_en'],dfm_embd_concat7['cs-en'],'t7-cs-en-5',5)

Iteration 1, loss = 0.35230331
Iteration 2, loss = 0.29980897
Iteration 3, loss = 0.28589394
Pearson: 0.2814258491367959
KT: 0.19809907214571246
Iteration 1, loss = 0.35230331
Iteration 2, loss = 0.29980897
Iteration 3, loss = 0.28589394
Iteration 4, loss = 0.27009908
Pearson: 0.2708989171474555
KT: 0.19196754862446294
Iteration 1, loss = 0.35230331
Iteration 2, loss = 0.29980897
Iteration 3, loss = 0.28589394
Iteration 4, loss = 0.27009908
Iteration 5, loss = 0.25766450
Pearson: 0.2693032919483958
KT: 0.19124666408244798


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

### de - en

In [668]:
model_pred1(df_embd_concat6['de_en'],dfm_embd_concat6['de-en'],'t6-de-en-3',3)
model_pred1(df_embd_concat6['de_en'],dfm_embd_concat6['de-en'],'t6-de-en-4',4)
model_pred1(df_embd_concat6['de_en'],dfm_embd_concat6['de-en'],'t6-de-en-5',5)

Iteration 1, loss = 0.37126481
Iteration 2, loss = 0.33871956
Iteration 3, loss = 0.33042529
Pearson: 0.44138546741246265
KT: 0.2999596576679815
Iteration 1, loss = 0.37126481
Iteration 2, loss = 0.33871956
Iteration 3, loss = 0.33042529
Iteration 4, loss = 0.32365799
Pearson: 0.42315880220330515
KT: 0.289004055671204
Iteration 1, loss = 0.37126481
Iteration 2, loss = 0.33871956
Iteration 3, loss = 0.33042529
Iteration 4, loss = 0.32365799
Iteration 5, loss = 0.31775204
Pearson: 0.41702592954378703
KT: 0.2966881931828327


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

In [669]:
model_pred1(df_embd_concat7['de_en'],dfm_embd_concat7['de-en'],'t7-de-en-3',3)
model_pred1(df_embd_concat7['de_en'],dfm_embd_concat7['de-en'],'t7-de-en-4',4)
model_pred1(df_embd_concat7['de_en'],dfm_embd_concat7['de-en'],'t7-de-en-5',5)

Iteration 1, loss = 0.36626644
Iteration 2, loss = 0.33801744
Iteration 3, loss = 0.32760127
Pearson: 0.417346181809636
KT: 0.28889193693731885
Iteration 1, loss = 0.36626644
Iteration 2, loss = 0.33801744
Iteration 3, loss = 0.32760127
Iteration 4, loss = 0.32032745
Pearson: 0.42523823118807186
KT: 0.29455393299851895
Iteration 1, loss = 0.36626644
Iteration 2, loss = 0.33801744
Iteration 3, loss = 0.32760127
Iteration 4, loss = 0.32032745
Iteration 5, loss = 0.31328107
Pearson: 0.4263181366208362
KT: 0.2990707162778921


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

### ru - en

In [670]:
model_pred1(df_embd_concat6['ru_en'],dfm_embd_concat6['ru-en'],'t6-ru-en-3',3)
model_pred1(df_embd_concat6['ru_en'],dfm_embd_concat6['ru-en'],'t6-ru-en-4',4)
model_pred1(df_embd_concat6['ru_en'],dfm_embd_concat6['ru-en'],'t6-ru-en-5',5)

Iteration 1, loss = 0.38584007
Iteration 2, loss = 0.35452077
Iteration 3, loss = 0.34572181
Pearson: 0.2844705718170658
KT: 0.21291761380105045
Iteration 1, loss = 0.38584007
Iteration 2, loss = 0.35452077
Iteration 3, loss = 0.34572181
Iteration 4, loss = 0.33760968
Pearson: 0.2666150475838711
KT: 0.2060793487795847
Iteration 1, loss = 0.38584007
Iteration 2, loss = 0.35452077
Iteration 3, loss = 0.34572181
Iteration 4, loss = 0.33760968
Iteration 5, loss = 0.33035231
Pearson: 0.267975224759319
KT: 0.20671229891045267


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

In [671]:
model_pred1(df_embd_concat7['ru_en'],dfm_embd_concat7['ru-en'],'t7-ru-en-3',3)

Iteration 1, loss = 0.38216677
Iteration 2, loss = 0.35217405
Iteration 3, loss = 0.34247712
Pearson: 0.287544510807454
KT: 0.21290158974710444
Pearson: 0.18108239098174134
KT: 0.12879133058442455
Pearson: 0.1740520201812954
KT: 0.10737918849904997


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

In [678]:
model_pred1(df_embd_concat7['ru_en'],dfm_embd_concat7['ru-en'],'t7-ru-en-4',4)
model_pred1(df_embd_concat7['ru_en'],dfm_embd_concat7['ru-en'],'t7-ru-en-5',5)

Pearson: 0.26797535985586923
KT: 0.1845852681919366
Pearson: 0.1932601628869458
KT: 0.12010629334567954


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

### zh - en

In [679]:
model_pred1(df_embd_concat6['zh_en'],dfm_embd_concat6['zh-en'],'t6-zh-en-3',3)
model_pred1(df_embd_concat6['zh_en'],dfm_embd_concat6['zh-en'],'t6-zh-en-4',4)
model_pred1(df_embd_concat6['zh_en'],dfm_embd_concat6['zh-en'],'t6-zh-en-5',5)

Iteration 1, loss = 0.36897910
Iteration 2, loss = 0.34159444
Iteration 3, loss = 0.33479619
Pearson: 0.2520449649741617
KT: 0.19072515798183745
Iteration 1, loss = 0.36897910
Iteration 2, loss = 0.34159444
Iteration 3, loss = 0.33479619
Iteration 4, loss = 0.32796899
Pearson: 0.23126212868307056
KT: 0.17472504125676835
Iteration 1, loss = 0.36897910
Iteration 2, loss = 0.34159444
Iteration 3, loss = 0.33479619
Iteration 4, loss = 0.32796899
Iteration 5, loss = 0.32266089
Pearson: 0.24648527401668333
KT: 0.1865979052328214


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

In [673]:
model_pred1(df_embd_concat7['zh_en'],dfm_embd_concat7['zh-en'],'t7-zh-en-3',3)
model_pred1(df_embd_concat7['zh_en'],dfm_embd_concat7['zh-en'],'t7-zh-en-4',4)
model_pred1(df_embd_concat7['zh_en'],dfm_embd_concat7['zh-en'],'t7-zh-en-5',5)

Pearson: 0.11835060792751342
KT: 0.045614156959635416
Pearson: 0.158492598671885
KT: 0.09158694413386981
Pearson: 0.15542757490798276
KT: 0.10712224404448661


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

### en - fi

In [680]:
model_pred1(df_embd_concat6['en_fi'],dfm_embd_concat6['en-fi'],'t6-en-fi-3',3)
model_pred1(df_embd_concat6['en_fi'],dfm_embd_concat6['en-fi'],'t6-en-fi-4',4)
model_pred1(df_embd_concat6['en_fi'],dfm_embd_concat6['en-fi'],'t6-en-fi-5',5)

Iteration 1, loss = 0.30877163
Iteration 2, loss = 0.25933534
Iteration 3, loss = 0.24058011
Pearson: 0.35155878581848227
KT: 0.23262380161959034
Iteration 1, loss = 0.30877163
Iteration 2, loss = 0.25933534
Iteration 3, loss = 0.24058011
Iteration 4, loss = 0.23372896
Pearson: 0.3414106211969802
KT: 0.22391755423898368
Iteration 1, loss = 0.30877163
Iteration 2, loss = 0.25933534
Iteration 3, loss = 0.24058011
Iteration 4, loss = 0.23372896
Iteration 5, loss = 0.21991057
Pearson: 0.3518600352211349
KT: 0.23547515770744315


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

In [ ]:
model_pred1(df_embd_concat7['en_fi'],dfm_embd_concat7['en-fi'],'t7-en-fi-3',3)
model_pred1(df_embd_concat7['en_fi'],dfm_embd_concat7['en-fi'],'t7-en-fi-4',4)
model_pred1(df_embd_concat7['en_fi'],dfm_embd_concat7['en-fi'],'t7-en-fi-5',5)

### en - zh

In [681]:
model_pred1(df_embd_concat6['en_zh'],dfm_embd_concat6['en-zh'],'t6-en-zh-3',3)
model_pred1(df_embd_concat6['en_zh'],dfm_embd_concat6['en-zh'],'t6-en-zh-4',4)
model_pred1(df_embd_concat6['en_zh'],dfm_embd_concat6['en-zh'],'t6-en-zh-5',5)

Iteration 1, loss = 0.37465207
Iteration 2, loss = 0.33689111
Iteration 3, loss = 0.32803624
Pearson: 0.41596810425449937
KT: 0.27888386251756675
Iteration 1, loss = 0.37465207
Iteration 2, loss = 0.33689111
Iteration 3, loss = 0.32803624
Iteration 4, loss = 0.31603820
Pearson: 0.4207450335862333
KT: 0.27846124717809473
Iteration 1, loss = 0.37465207
Iteration 2, loss = 0.33689111
Iteration 3, loss = 0.32803624
Iteration 4, loss = 0.31603820
Iteration 5, loss = 0.30638135
Pearson: 0.40567046358618725
KT: 0.2664992206170399


MLPRegressor(activation='tanh', batch_size=16,
             hidden_layer_sizes=(3072, 1536, 768), learning_rate='adaptive',
             learning_rate_init=3e-05, max_iter=5, random_state=6,
             verbose=True)

In [ ]:
model_pred1(df_embd_concat7['en_zh'],dfm_embd_concat7['en-zh'],'t7-en-zh-3',3)
model_pred1(df_embd_concat7['en_zh'],dfm_embd_concat7['en-zh'],'t7-en-zh-4',4)
model_pred1(df_embd_concat7['en_zh'],dfm_embd_concat7['en-zh'],'t7-en-zh-5',5)

Iteration 1, loss = 0.38905685
Iteration 2, loss = 0.33636289
Iteration 3, loss = 0.32682515
Pearson: 0.41727421158889766
KT: 0.27693178214000563
Iteration 1, loss = 0.38905685
Iteration 2, loss = 0.33636289
